# SimFin Tutorial 10 - Neural Networks - grid_search

[Original repository on GitHub](https://github.com/simfin/simfin-tutorials)

This tutorial was originally written by [Hvass Labs](https://github.com/Hvass-Labs)

----

"The attackers can be stopped by removing the head or destroying the brain." &ndash; [Shaun of The Dead](https://www.youtube.com/watch?v=ONEZfj6h1xc)

## Introduction

In the previous tutorial we used a Machine Learning (ML) model to try and predict future stock-returns for 1-3 year investment periods given various signals. This worked quite well on the training-set, but unfortunately it failed to generalize to the test-set.

In this tutorial we will use a so-called Neural Network, which has become very fashionable in recent years, because they have been able to solve extremely hard problems, that were previously impossible to solve.

Unfortunately, Neural Networks have become so hyped, that people sometimes misunderstand what they really are: A Neural Network is just a function approximator that maps some input x (e.g. signals) to some output y (e.g. stock-returns). It consists of a number of layers of both linear and non-linear mappings, whose parameters will be learned during the training process, so it learns to map from x to y in the training-set, and the learned mapping hopefully generalizes to new data.

In theory, a Neural Network is capable of approximating any function to any desired degree of precision - but what is truly important, is not how well it performs on the training-set, but how well it performs on new data that it has not seen during the training.

This tutorial shows how to use financial data from SimFin with the TensorFlow library for creating Neural Networks. It is assumed you are already familiar with both.

## Imports

In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns

# TensorFlow and Keras imports.
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Dense, InputLayer
from tensorflow.keras.utils import to_categorical

# Scikit-learn imports.
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight

# Import the main functionality from the SimFin Python API.
import simfin as sf

# Import names used for easy access to SimFin's data-columns.
from simfin.names import *

In [3]:
import tensorflow.keras as keras

In [4]:
# Version of the SimFin Python API.
sf.__version__

'0.8.3'

In [5]:
# TensorFlow version.
tf.__version__

'2.6.0'

## SimFin Config

In [6]:
# SimFin data-directory.
sf.set_data_dir('~/simfin_data/')

In [7]:
# SimFin load API key or use free data.
sf.load_api_key(path='~/simfin_api_key.txt', default_key='free')

In [8]:
# Seaborn set plotting style.
sns.set_style("whitegrid")

## Data Hub

We use a `StockHub` object to easily load and process financial data with these settings:

In [9]:
# We are interested in the US stock-market.
market = 'us'

# Add this date-offset to the fundamental data such as
# Income Statements etc., because the REPORT_DATE is not
# when it was actually made available to the public,
# which can be 1, 2 or even 3 months after the Report Date.
offset = pd.DateOffset(days=60)

# Refresh the fundamental datasets (Income Statements etc.)
# every 30 days.
refresh_days = 30

# Refresh the dataset with shareprices every 10 days.
refresh_days_shareprices = 10

In [10]:
%%time
hub = sf.StockHub(market=market, offset=offset,
                  refresh_days=refresh_days,
                  refresh_days_shareprices=refresh_days_shareprices)

CPU times: user 9 µs, sys: 2 µs, total: 11 µs
Wall time: 13.4 µs


## Signals

First we calculate financial signals for the stocks, such as the Current Ratio, Debt Ratio, Net Profit Margin, Return on Assets, etc. These are calculated using data from the financial reports: Income Statements, Balance Sheets and Cash-Flow Statements, which are automatically downloaded and loaded by the data-hub.

In [11]:
%%time
df_fin_signals = hub.fin_signals(variant='daily')

Dataset "us-income-ttm" on disk (21 days old).
- Loading from disk ... Done!
Dataset "us-balance-ttm" on disk (20 days old).
- Loading from disk ... Done!
Dataset "us-cashflow-ttm" on disk (20 days old).
- Loading from disk ... Done!
Dataset "us-shareprices-daily" on disk (9 days old).
- Loading from disk ... Done!
Cache-file 'fin_signals-2a38bb7d.pickle' on disk (9 days old).
- Loading from disk ... Done!
CPU times: user 6.98 s, sys: 1.21 s, total: 8.19 s
Wall time: 8.7 s


Now we calculate growth signals for the stocks, such as Earnings Growth, FCF Growth, Sales Growth, etc. These are also calculated using data from the financial reports: Income Statements, Balance Sheets and Cash-Flow Statements, which are automatically downloaded and loaded by the data-hub.

In [12]:
%%time
df_growth_signals = hub.growth_signals(variant='daily')

Dataset "us-income-quarterly" on disk (20 days old).
- Loading from disk ... Done!
Dataset "us-balance-quarterly" on disk (20 days old).
- Loading from disk ... Done!
Dataset "us-cashflow-quarterly" on disk (20 days old).
- Loading from disk ... Done!
Cache-file 'growth_signals-2a38bb7d.pickle' on disk (9 days old).
- Loading from disk ... Done!
CPU times: user 696 ms, sys: 252 ms, total: 949 ms
Wall time: 949 ms


In [13]:
df_growth_signals

Assets Growth  Assets Growth QOQ  Assets Growth YOY  \
Ticker Date                                                              
A      2007-01-03            NaN                NaN                NaN   
       2007-01-04            NaN                NaN                NaN   
       2007-01-05            NaN                NaN                NaN   
       2007-01-08            NaN                NaN                NaN   
       2007-01-09            NaN                NaN                NaN   
...                          ...                ...                ...   
ZYXI   2020-09-24       0.683181           0.106466           0.683181   
       2020-09-25       0.683181           0.106466           0.683181   
       2020-09-28       0.683181           0.106466           0.683181   
       2020-09-29       0.683181           0.106466           0.683181   
       2020-09-30       0.683181           0.106466           0.683181   

                   Earnings Growth  Earnings Growth QOQ  Earnings Growth YOY  \
Ticker Date                                                                    
A      2007-01-03              NaN                  NaN                  NaN   
       2007-01-04              NaN                  NaN                  NaN   
       2007-01-05              NaN                  NaN                  NaN   
       2007-01-08              NaN                  NaN                  NaN   
       2007-01-09              NaN                  NaN                  NaN   
...                            ...                  ...                  ...   
ZYXI   2020-09-24         0.124203             0.027239             0.395467   
       2020-09-25         0.124203             0.027239             0.395467   
       2020-09-28         0.124203             0.027239             0.395467   
       2020-09-29         0.124203             0.027239             0.395467   
       2020-09-30         0.124203             0.027239             0.395467   

                   FCF Growth  FCF Growth QOQ  FCF Growth YOY  Sales Growth  \
Ticker Date                                                                   
A      2007-01-03         NaN             NaN             NaN           NaN   
       2007-01-04         NaN             NaN             NaN           NaN   
       2007-01-05         NaN             NaN             NaN           NaN   
       2007-01-08         NaN             NaN             NaN           NaN   
       2007-01-09         NaN             NaN             NaN           NaN   
...                       ...             ...             ...           ...   
ZYXI   2020-09-24   -0.212002        5.847761        2.823333      0.636049   
       2020-09-25   -0.212002        5.847761        2.823333      0.636049   
       2020-09-28   -0.212002        5.847761        2.823333      0.636049   
       2020-09-29   -0.212002        5.847761        2.823333      0.636049   
       2020-09-30   -0.212002        5.847761        2.823333      0.636049   

                   Sales Growth QOQ  Sales Growth YOY  
Ticker Date                                            
A      2007-01-03               NaN               NaN  
       2007-01-04               NaN               NaN  
       2007-01-05               NaN               NaN  
       2007-01-08               NaN               NaN  
       2007-01-09               NaN               NaN  
...                             ...               ...  
ZYXI   2020-09-24          0.264972          0.870739  
       2020-09-25          0.264972          0.870739  
       2020-09-28          0.264972          0.870739  
       2020-09-29          0.264972          0.870739  
       2020-09-30          0.264972          0.870739  

[7104243 rows x 12 columns]

Now we calculate valuation signals for the stocks, such as P/E, P/Sales, etc. These are calculated from the share-prices and data from the financial reports. Because the data-hub has already loaded the required datasets in the function-calls above, the data is merely reused here, and the data-hub can proceed directly to computing the signals.

In [14]:
%%time
df_val_signals = hub.val_signals(variant='daily')

Cache-file 'val_signals-739b68a6.pickle' on disk (9 days old).
- Loading from disk ... Done!
CPU times: user 19.4 ms, sys: 221 ms, total: 240 ms
Wall time: 238 ms


We now combine all the signals into a single DataFrame:

In [15]:
%%time
# Combine the DataFrames.
dfs = [df_fin_signals, df_growth_signals, df_val_signals]
df_signals = pd.concat(dfs, axis=1)

CPU times: user 2.88 s, sys: 584 ms, total: 3.47 s
Wall time: 3.47 s


We can then show the results:

In [16]:
df_signals.dropna(how='all').head()

(Dividends + Share Buyback) / FCF  Asset Turnover  \
Ticker Date                                                            
A      2010-09-29                           0.171492        0.553297   
       2010-09-30                           0.171492        0.553297   
       2010-10-01                           0.171492        0.553297   
       2010-10-04                           0.171492        0.553297   
       2010-10-05                           0.171492        0.553297   

                   CapEx / (Depr + Amor)  Current Ratio  Debt Ratio  \
Ticker Date                                                           
A      2010-09-29               0.622857       1.966061    0.404176   
       2010-09-30               0.622857       1.966061    0.404176   
       2010-10-01               0.622857       1.966061    0.404176   
       2010-10-04               0.622857       1.966061    0.404176   
       2010-10-05               0.622857       1.966061    0.404176   

                   Dividends / FCF  Gross Profit Margin  Interest Coverage  \
Ticker Date                                                                  
A      2010-09-29             -0.0             0.542205           5.636364   
       2010-09-30             -0.0             0.542205           5.636364   
       2010-10-01             -0.0             0.542205           5.636364   
       2010-10-04             -0.0             0.542205           5.636364   
       2010-10-05             -0.0             0.542205           5.636364   

                   Inventory Turnover  Log Revenue  ...  Earnings Yield  \
Ticker Date                                         ...                   
A      2010-09-29            7.318314     9.701999  ...        0.036491   
       2010-09-30            7.318314     9.701999  ...        0.035627   
       2010-10-01            7.318314     9.701999  ...        0.035247   
       2010-10-04            7.318314     9.701999  ...        0.036480   
       2010-10-05            7.318314     9.701999  ...        0.035983   

                   FCF Yield    Market-Cap    P/Cash        P/E      P/FCF  \
Ticker Date                                                                  
A      2010-09-29   0.039291  1.142744e+10  4.931996  27.403921  25.450857   
       2010-09-30   0.038361  1.170453e+10  5.051587  28.068411  26.067990   
       2010-10-01   0.037952  1.183080e+10  5.106084  28.371217  26.349215   
       2010-10-04   0.039279  1.143094e+10  4.933510  27.412332  25.458669   
       2010-10-05   0.038744  1.158878e+10  5.001631  27.790839  25.810200   

                      P/NCAV  P/NetNet   P/Sales  Price to Book Value  
Ticker Date                                                            
A      2010-09-29 -20.589973 -3.763357  2.269600             4.066703  
       2010-09-30 -21.089239 -3.854611  2.324633             4.165312  
       2010-10-01 -21.316752 -3.896195  2.349712             4.210248  
       2010-10-04 -20.596293 -3.764513  2.270296             4.067951  
       2010-10-05 -20.880685 -3.816493  2.301644             4.124121  

[5 rows x 42 columns]

In [17]:
df_signals.shape[0]

7104243

## Missing Data

Some of the signals have a lot of missing data which cannot be handled by TensorFlow. Let us first see the fraction of each signal-column that is missing:

In [18]:
# Remove all rows with only NaN values.
df = df_signals.dropna(how='all').reset_index(drop=True)

# For each column, show the fraction of the rows that are NaN.
(df.isnull().sum() / len(df)).sort_values(ascending=False)

R&D / Gross Profit                   0.646990
Return on Research Capital           0.646990
R&D / Revenue                        0.612017
Dividend Yield                       0.441062
Inventory Turnover                   0.288254
Debt Ratio                           0.262483
Net Acquisitions / Total Assets      0.258719
Sales Growth                         0.147160
Sales Growth YOY                     0.144672
FCF Growth                           0.135036
Earnings Growth                      0.135036
Assets Growth                        0.135036
FCF Growth YOY                       0.133256
Earnings Growth YOY                  0.133241
Assets Growth YOY                    0.133241
Interest Coverage                    0.097762
Gross Profit Margin                  0.093035
Sales Growth QOQ                     0.053912
Assets Growth QOQ                    0.040347
Earnings Growth QOQ                  0.040347
FCF Growth QOQ                       0.040347
CapEx / (Depr + Amor)             

In [19]:
df

,(Dividends + Share Buyback) / FCF,Asset Turnover,CapEx / (Depr + Amor),Current Ratio,Debt Ratio,Dividends / FCF,Gross Profit Margin,Interest Coverage,Inventory Turnover,Log Revenue,...,Earnings Yield,FCF Yield,Market-Cap,P/Cash,P/E,P/FCF,P/NCAV,P/NetNet,P/Sales,Price to Book Value
0,0.171492,0.553297,0.622857,1.966061,0.404176,-0.000000,0.542205,5.636364,7.318314,9.701999,...,0.036491,0.039291,1.142744e+10,4.931996,27.403921,25.450857,-20.589973,-3.763357,2.269600,4.066703
1,0.171492,0.553297,0.622857,1.966061,0.404176,-0.000000,0.542205,5.636364,7.318314,9.701999,...,0.035627,0.038361,1.170453e+10,5.051587,28.068411,26.067990,-21.089239,-3.854611,2.324633,4.165312
2,0.171492,0.553297,0.622857,1.966061,0.404176,-0.000000,0.542205,5.636364,7.318314,9.701999,...,0.035247,0.037952,1.183080e+10,5.106084,28.371217,26.349215,-21.316752,-3.896195,2.349712,4.210248
3,0.171492,0.553297,0.622857,1.966061,0.404176,-0.000000,0.542205,5.636364,7.318314,9.701999,...,0.036480,0.039279,1.143094e+10,4.933510,27.412332,25.458669,-20.596293,-3.764513,2.270296,4.067951
4,0.171492,0.553297,0.622857,1.966061,0.404176,-0.000000,0.542205,5.636364,7.318314,9.701999,...,0.035983,0.038744,1.158878e+10,5.001631,27.790839,25.810200,-20.880685,-3.816493,2.301644,4.124121
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4255107,-0.055400,1.645039,NaN,4.955648,NaN,0.000468,0.792558,912.071429,12.846824,7.781540,...,0.020064,0.011759,5.449494e+08,32.215025,49.839891,85.042037,27.205300,36.535776,9.011896,20.215505
4255108,-0.055400,1.645039,NaN,4.955648,NaN,0.000468,0.792558,912.071429,12.846824,7.781540,...,0.019139,0.011217,5.712903e+08,33.772186,52.248976,89.152669,28.520309,38.301787,9.447500,21.192651
4255109,-0.055400,1.645039,NaN,4.955648,NaN,0.000468,0.792558,912.071429,12.846824,7.781540,...,0.018913,0.011084,5.781321e+08,34.176643,52.874712,90.220365,28.861869,38.760491,9.560643,21.446455
4255110,-0.055400,1.645039,NaN,4.955648,NaN,0.000468,0.792558,912.071429,12.846824,7.781540,...,0.018735,0.010980,5.836055e+08,34.500209,53.375301,91.074522,29.135118,39.127454,9.651158,21.649499


Let us remove all signals that have more than 25% missing data:

In [20]:
# List of the columns before removing any.
columns_before = df_signals.columns

# Threshold for the number of rows that must be NaN for each column.
thresh = 0.75 * len(df_signals.dropna(how='all'))

# Remove all columns which don't have sufficient data.
df_signals = df_signals.dropna(axis='columns', thresh=thresh)

# List of the columns after the removal.
columns_after = df_signals.columns

# Show the columns that were removed.
columns_before.difference(columns_after)

Index(['Debt Ratio', 'Dividend Yield', 'Inventory Turnover',
       'Net Acquisitions / Total Assets', 'R&D / Gross Profit',
       'R&D / Revenue', 'Return on Research Capital'],
      dtype='object')

In [21]:
# print all columns
with pd.option_context('display.max_rows', 6, 'display.max_columns', None): 
    display(df_signals)

(Dividends + Share Buyback) / FCF  Asset Turnover  \
Ticker Date                                                            
A      2007-01-03                                NaN             NaN   
       2007-01-04                                NaN             NaN   
       2007-01-05                                NaN             NaN   
...                                              ...             ...   
ZYXI   2020-09-28                            -0.0554        1.645039   
       2020-09-29                            -0.0554        1.645039   
       2020-09-30                            -0.0554        1.645039   

                   CapEx / (Depr + Amor)  Current Ratio  Dividends / FCF  \
Ticker Date                                                                
A      2007-01-03                    NaN            NaN              NaN   
       2007-01-04                    NaN            NaN              NaN   
       2007-01-05                    NaN            NaN              NaN   
...                                  ...            ...              ...   
ZYXI   2020-09-28                    NaN       4.955648         0.000468   
       2020-09-29                    NaN       4.955648         0.000468   
       2020-09-30                    NaN       4.955648         0.000468   

                   Gross Profit Margin  Interest Coverage  Log Revenue  \
Ticker Date                                                              
A      2007-01-03                  NaN                NaN          NaN   
       2007-01-04                  NaN                NaN          NaN   
       2007-01-05                  NaN                NaN          NaN   
...                                ...                ...          ...   
ZYXI   2020-09-28             0.792558         912.071429      7.78154   
       2020-09-29             0.792558         912.071429      7.78154   
       2020-09-30             0.792558         912.071429      7.78154   

                   Net Profit Margin  Quick Ratio  Return on Assets  \
Ticker Date                                                           
A      2007-01-03                NaN          NaN               NaN   
       2007-01-04                NaN          NaN               NaN   
       2007-01-05                NaN          NaN               NaN   
...                              ...          ...               ...   
ZYXI   2020-09-28           0.180817     4.016611          0.297451   
       2020-09-29           0.180817     4.016611          0.297451   
       2020-09-30           0.180817     4.016611          0.297451   

                   Return on Equity  Share Buyback / FCF  Assets Growth  \
Ticker Date                                                               
A      2007-01-03               NaN                  NaN            NaN   
       2007-01-04               NaN                  NaN            NaN   
       2007-01-05               NaN                  NaN            NaN   
...                             ...                  ...            ...   
ZYXI   2020-09-28          0.405609            -0.055868       0.683181   
       2020-09-29          0.405609            -0.055868       0.683181   
       2020-09-30          0.405609            -0.055868       0.683181   

                   Assets Growth QOQ  Assets Growth YOY  Earnings Growth  \
Ticker Date                                                                
A      2007-01-03                NaN                NaN              NaN   
       2007-01-04                NaN                NaN              NaN   
       2007-01-05                NaN                NaN              NaN   
...                              ...                ...              ...   
ZYXI   2020-09-28           0.106466           0.683181         0.124203   
       2020-09-29           0.106466           0.683181         0.124203   
       2020-09-30           0.106466           0.683181         0.124203   

                 

## 1-3 Year Returns

We want to try and predict the average 1-3 year returns. We will actually consider the mean log-returns, because that is easier to calculate, but the result is nearly the same as the non-log mean returns.

In [22]:
# Name of the new column for the returns.
# TOTAL_RETURN_1_3Y = 'Total Return 1-3 Years'

In [23]:
# %%time
# Calculate the mean log-returns for all 1-3 year periods.
# df_returns_1_3y = \
#     hub.mean_log_returns(name=TOTAL_RETURN_1_3Y,
#                          future=True, annualized=True,
#                          min_years=1, max_years=3)

## Top price within 1 year

average of top 10 prices of next year

In [24]:
df_fin_signals.head()

(Dividends + Share Buyback) / FCF  Asset Turnover  \
Ticker Date                                                            
A      2007-01-03                                NaN             NaN   
       2007-01-04                                NaN             NaN   
       2007-01-05                                NaN             NaN   
       2007-01-08                                NaN             NaN   
       2007-01-09                                NaN             NaN   

                   CapEx / (Depr + Amor)  Current Ratio  Debt Ratio  \
Ticker Date                                                           
A      2007-01-03                    NaN            NaN         NaN   
       2007-01-04                    NaN            NaN         NaN   
       2007-01-05                    NaN            NaN         NaN   
       2007-01-08                    NaN            NaN         NaN   
       2007-01-09                    NaN            NaN         NaN   

                   Dividends / FCF  Gross Profit Margin  Interest Coverage  \
Ticker Date                                                                  
A      2007-01-03              NaN                  NaN                NaN   
       2007-01-04              NaN                  NaN                NaN   
       2007-01-05              NaN                  NaN                NaN   
       2007-01-08              NaN                  NaN                NaN   
       2007-01-09              NaN                  NaN                NaN   

                   Inventory Turnover  Log Revenue  \
Ticker Date                                          
A      2007-01-03                 NaN          NaN   
       2007-01-04                 NaN          NaN   
       2007-01-05                 NaN          NaN   
       2007-01-08                 NaN          NaN   
       2007-01-09                 NaN          NaN   

                   Net Acquisitions / Total Assets  Net Profit Margin  \
Ticker Date                                                             
A      2007-01-03                              NaN                NaN   
       2007-01-04                              NaN                NaN   
       2007-01-05                              NaN                NaN   
       2007-01-08                              NaN                NaN   
       2007-01-09                              NaN                NaN   

                   Quick Ratio  R&D / Gross Profit  R&D / Revenue  \
Ticker Date                                                         
A      2007-01-03          NaN                 NaN            NaN   
       2007-01-04          NaN                 NaN            NaN   
       2007-01-05          NaN                 NaN            NaN   
       2007-01-08          NaN                 NaN            NaN   
       2007-01-09          NaN                 NaN            NaN   

                   Return on Assets  Return on Equity  \
Ticker Date                                             
A      2007-01-03               NaN               NaN   
       2007-01-04               NaN               NaN   
       2007-01-05               NaN               NaN   
       2007-01-08               NaN               NaN   
       2007-01-09               NaN               NaN   

                   Return on Research Capital  Share Buyback / FCF  
Ticker Date                                                         
A      2007-01-03                         NaN                  NaN  
       2007-01-04                         NaN                  NaN  
       2007-01-05                         NaN                  NaN  
       2007-01-08                         NaN                  NaN  
       2007-01-09                         NaN                  NaN

In [25]:
df_growth_signals

Assets Growth  Assets Growth QOQ  Assets Growth YOY  \
Ticker Date                                                              
A      2007-01-03            NaN                NaN                NaN   
       2007-01-04            NaN                NaN                NaN   
       2007-01-05            NaN                NaN                NaN   
       2007-01-08            NaN                NaN                NaN   
       2007-01-09            NaN                NaN                NaN   
...                          ...                ...                ...   
ZYXI   2020-09-24       0.683181           0.106466           0.683181   
       2020-09-25       0.683181           0.106466           0.683181   
       2020-09-28       0.683181           0.106466           0.683181   
       2020-09-29       0.683181           0.106466           0.683181   
       2020-09-30       0.683181           0.106466           0.683181   

                   Earnings Growth  Earnings Growth QOQ  Earnings Growth YOY  \
Ticker Date                                                                    
A      2007-01-03              NaN                  NaN                  NaN   
       2007-01-04              NaN                  NaN                  NaN   
       2007-01-05              NaN                  NaN                  NaN   
       2007-01-08              NaN                  NaN                  NaN   
       2007-01-09              NaN                  NaN                  NaN   
...                            ...                  ...                  ...   
ZYXI   2020-09-24         0.124203             0.027239             0.395467   
       2020-09-25         0.124203             0.027239             0.395467   
       2020-09-28         0.124203             0.027239             0.395467   
       2020-09-29         0.124203             0.027239             0.395467   
       2020-09-30         0.124203             0.027239             0.395467   

                   FCF Growth  FCF Growth QOQ  FCF Growth YOY  Sales Growth  \
Ticker Date                                                                   
A      2007-01-03         NaN             NaN             NaN           NaN   
       2007-01-04         NaN             NaN             NaN           NaN   
       2007-01-05         NaN             NaN             NaN           NaN   
       2007-01-08         NaN             NaN             NaN           NaN   
       2007-01-09         NaN             NaN             NaN           NaN   
...                       ...             ...             ...           ...   
ZYXI   2020-09-24   -0.212002        5.847761        2.823333      0.636049   
       2020-09-25   -0.212002        5.847761        2.823333      0.636049   
       2020-09-28   -0.212002        5.847761        2.823333      0.636049   
       2020-09-29   -0.212002        5.847761        2.823333      0.636049   
       2020-09-30   -0.212002        5.847761        2.823333      0.636049   

                   Sales Growth QOQ  Sales Growth YOY  
Ticker Date                                            
A      2007-01-03               NaN               NaN  
       2007-01-04               NaN               NaN  
       2007-01-05               NaN               NaN  
       2007-01-08               NaN               NaN  
       2007-01-09               NaN               NaN  
...                             ...               ...  
ZYXI   2020-09-24          0.264972          0.870739  
       2020-09-25          0.264972          0.870739  
       2020-09-28          0.264972          0.870739  
       2020-09-29          0.264972          0.870739  
       2020-09-30          0.264972          0.870739  

[7104243 rows x 12 columns]

In [26]:
df_val_signals

Dividend Yield  Earnings Yield  FCF Yield   Market-Cap  \
Ticker Date                                                                 
A      2007-01-03             NaN             NaN        NaN          NaN   
       2007-01-04             NaN             NaN        NaN          NaN   
       2007-01-05             NaN             NaN        NaN          NaN   
       2007-01-08             NaN             NaN        NaN          NaN   
       2007-01-09             NaN             NaN        NaN          NaN   
...                           ...             ...        ...          ...   
ZYXI   2020-09-24        0.000006        0.020064   0.011759  544949370.0   
       2020-09-25        0.000005        0.019139   0.011217  571290300.0   
       2020-09-28        0.000005        0.018913   0.011084  578132100.0   
       2020-09-29        0.000005        0.018735   0.010980  583605540.0   
       2020-09-30        0.000005        0.018317   0.010735  596947050.0   

                      P/Cash        P/E      P/FCF     P/NCAV   P/NetNet  \
Ticker Date                                                                
A      2007-01-03        NaN        NaN        NaN        NaN        NaN   
       2007-01-04        NaN        NaN        NaN        NaN        NaN   
       2007-01-05        NaN        NaN        NaN        NaN        NaN   
       2007-01-08        NaN        NaN        NaN        NaN        NaN   
       2007-01-09        NaN        NaN        NaN        NaN        NaN   
...                      ...        ...        ...        ...        ...   
ZYXI   2020-09-24  32.215025  49.839891  85.042037  27.205300  36.535776   
       2020-09-25  33.772186  52.248976  89.152669  28.520309  38.301787   
       2020-09-28  34.176643  52.874712  90.220365  28.861869  38.760491   
       2020-09-29  34.500209  53.375301  91.074522  29.135118  39.127454   
       2020-09-30  35.288901  54.595487  93.156531  29.801161  40.021927   

                    P/Sales  Price to Book Value  
Ticker Date                                       
A      2007-01-03       NaN                  NaN  
       2007-01-04       NaN                  NaN  
       2007-01-05       NaN                  NaN  
       2007-01-08       NaN                  NaN  
       2007-01-09       NaN                  NaN  
...                     ...                  ...  
ZYXI   2020-09-24  9.011896            20.215505  
       2020-09-25  9.447500            21.192651  
       2020-09-28  9.560643            21.446455  
       2020-09-29  9.651158            21.649499  
       2020-09-30  9.871788            22.144417  

[7104243 rows x 11 columns]

In [27]:
df_signals

(Dividends + Share Buyback) / FCF  Asset Turnover  \
Ticker Date                                                            
A      2007-01-03                                NaN             NaN   
       2007-01-04                                NaN             NaN   
       2007-01-05                                NaN             NaN   
       2007-01-08                                NaN             NaN   
       2007-01-09                                NaN             NaN   
...                                              ...             ...   
ZYXI   2020-09-24                            -0.0554        1.645039   
       2020-09-25                            -0.0554        1.645039   
       2020-09-28                            -0.0554        1.645039   
       2020-09-29                            -0.0554        1.645039   
       2020-09-30                            -0.0554        1.645039   

                   CapEx / (Depr + Amor)  Current Ratio  Dividends / FCF  \
Ticker Date                                                                
A      2007-01-03                    NaN            NaN              NaN   
       2007-01-04                    NaN            NaN              NaN   
       2007-01-05                    NaN            NaN              NaN   
       2007-01-08                    NaN            NaN              NaN   
       2007-01-09                    NaN            NaN              NaN   
...                                  ...            ...              ...   
ZYXI   2020-09-24                    NaN       4.955648         0.000468   
       2020-09-25                    NaN       4.955648         0.000468   
       2020-09-28                    NaN       4.955648         0.000468   
       2020-09-29                    NaN       4.955648         0.000468   
       2020-09-30                    NaN       4.955648         0.000468   

                   Gross Profit Margin  Interest Coverage  Log Revenue  \
Ticker Date                                                              
A      2007-01-03                  NaN                NaN          NaN   
       2007-01-04                  NaN                NaN          NaN   
       2007-01-05                  NaN                NaN          NaN   
       2007-01-08                  NaN                NaN          NaN   
       2007-01-09                  NaN                NaN          NaN   
...                                ...                ...          ...   
ZYXI   2020-09-24             0.792558         912.071429      7.78154   
       2020-09-25             0.792558         912.071429      7.78154   
       2020-09-28             0.792558         912.071429      7.78154   
       2020-09-29             0.792558         912.071429      7.78154   
       2020-09-30             0.792558         912.071429      7.78154   

                   Net Profit Margin  Quick Ratio  ...  Earnings Yield  \
Ticker Date                                        ...                   
A      2007-01-03                NaN          NaN  ...             NaN   
       2007-01-04                NaN          NaN  ...             NaN   
       2007-01-05                NaN          NaN  ...             NaN   
       2007-01-08                NaN          NaN  ...             NaN   
       2007-01-09                NaN          NaN  ...             NaN   
...                              ...          ...  ...             ...   
ZYXI   2020-09-24           0.180817     4.016611  ...        0.020064   
       2020-09-25           0.180817     4.016611  ...        0.019139   
       2020-09-28           0.180817     4.016611  ...        0.018913   
       2020-09-29           0.180817     4.016611  ...        0.018735   
       2020-09-30           0.180817     4.016611  ...        0.018317   

                   FCF Yield   Market-Cap     P/Cash        P/E      P/FCF  \
Ticker Date                                                                  
A      2007

In [28]:
%%time
df_prices = hub.load_shareprices(variant='daily')

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 6.44 µs


In [29]:
df_prices.head()

SimFinId   Open    Low   High  Close  Adj. Close  Dividend  \
Ticker Date                                                                     
A      2007-01-03     45846  34.99  34.05  35.48  34.30       22.66       NaN   
       2007-01-04     45846  34.30  33.46  34.60  34.41       22.73       NaN   
       2007-01-05     45846  34.30  34.00  34.40  34.09       22.52       NaN   
       2007-01-08     45846  33.98  33.68  34.08  33.97       22.44       NaN   
       2007-01-09     45846  34.08  33.63  34.32  34.01       22.47       NaN   

                    Volume  Shares Outstanding  
Ticker Date                                     
A      2007-01-03  2574600                 NaN  
       2007-01-04  2073700                 NaN  
       2007-01-05  2676600                 NaN  
       2007-01-08  1557200                 NaN  
       2007-01-09  1386200                 NaN

In [30]:
# https://stackoverflow.com/questions/53927460/select-rows-in-pandas-multiindex-dataframe
df_prices.index

MultiIndex([(   'A', '2007-01-03'),
            (   'A', '2007-01-04'),
            (   'A', '2007-01-05'),
            (   'A', '2007-01-08'),
            (   'A', '2007-01-09'),
            (   'A', '2007-01-10'),
            (   'A', '2007-01-11'),
            (   'A', '2007-01-12'),
            (   'A', '2007-01-16'),
            (   'A', '2007-01-17'),
            ...
            ('ZYXI', '2020-09-17'),
            ('ZYXI', '2020-09-18'),
            ('ZYXI', '2020-09-21'),
            ('ZYXI', '2020-09-22'),
            ('ZYXI', '2020-09-23'),
            ('ZYXI', '2020-09-24'),
            ('ZYXI', '2020-09-25'),
            ('ZYXI', '2020-09-28'),
            ('ZYXI', '2020-09-29'),
            ('ZYXI', '2020-09-30')],
           names=['Ticker', 'Date'], length=7104243)

In [31]:
# https://stackoverflow.com/questions/53927460/select-rows-in-pandas-multiindex-dataframe
df_prices.loc['NFLX']
df_prices[df_prices.index.get_level_values("Ticker").isin(['NFLX', 'A'])]
# df_prices.index.get_level_values("Ticker").isin(['NFLX', 'A'])
df_prices[df_prices.index.get_level_values("Ticker").isin(['NFLX', 'A'])]
df_prices[df_prices.index.get_level_values('Date').isin(['2020-09-30', '2020-09-29'])]
df_prices.loc[('NFLX', '2020-09-30'), :]

,,SimFinId,Open,Low,High,Close,Adj. Close,Dividend,Volume,Shares Outstanding
Ticker,Date,,,,,,,,,
NFLX,2020-09-30,83548,492.57,489.27,504.63,500.03,500.03,NaN,4634111,441795008.0


## Top price within 1 year

In [32]:
#https://stackoverflow.com/questions/43921322/find-highest-value-among-the-next-k-consecutive-rows-in-pandas
#https://stackoverflow.com/questions/42119793/doing-a-groupby-and-rolling-window-on-a-pandas-dataframe-with-a-multilevel-index

#%%time
max_price_1y = 'Top price average in 1 Year'
# Calculate the average of top 10 prices of next 1 year.
# df_top_average_1y = df_prices.nlargest(3,'Close')

# Calculate the top 1 Close prices of next 1 year. for each Ticker
#df_top_1y = df_prices.groupby(level="Ticker").Close.rolling(365).max().shift(-365)
df_top_1y = df_prices.groupby(level="Ticker").Close.apply(lambda x: x.rolling(window=365).max().shift(-365))

In [33]:
df_top_1y

Ticker  Date      
A       2007-01-03    40.4
        2007-01-04    40.4
        2007-01-05    40.4
        2007-01-08    40.4
        2007-01-09    40.4
                      ... 
ZYXI    2020-09-24     NaN
        2020-09-25     NaN
        2020-09-28     NaN
        2020-09-29     NaN
        2020-09-30     NaN
Name: Close, Length: 7104243, dtype: float64

In [34]:
df_prices2=pd.concat([df_prices, df_top_1y], axis = 1) 

In [35]:
# Changing columns name with index number
df_prices2.columns.values[9]= "Close_1y"

In [36]:
df_prices2

SimFinId   Open    Low   High  Close  Adj. Close  Dividend  \
Ticker Date                                                                     
A      2007-01-03     45846  34.99  34.05  35.48  34.30       22.66       NaN   
       2007-01-04     45846  34.30  33.46  34.60  34.41       22.73       NaN   
       2007-01-05     45846  34.30  34.00  34.40  34.09       22.52       NaN   
       2007-01-08     45846  33.98  33.68  34.08  33.97       22.44       NaN   
       2007-01-09     45846  34.08  33.63  34.32  34.01       22.47       NaN   
...                     ...    ...    ...    ...    ...         ...       ...   
ZYXI   2020-09-24    171401  16.47  15.85  16.68  15.93       15.93       NaN   
       2020-09-25    171401  15.89  15.86  16.78  16.70       16.70       NaN   
       2020-09-28    171401  16.92  16.30  17.17  16.90       16.90       NaN   
       2020-09-29    171401  16.86  16.80  17.20  17.06       17.06       NaN   
       2020-09-30    171401  17.11  17.04  17.88  17.45       17.45       NaN   

                    Volume  Shares Outstanding  Close_1y  
Ticker Date                                               
A      2007-01-03  2574600                 NaN      40.4  
       2007-01-04  2073700                 NaN      40.4  
       2007-01-05  2676600                 NaN      40.4  
       2007-01-08  1557200                 NaN      40.4  
       2007-01-09  1386200                 NaN      40.4  
...                    ...                 ...       ...  
ZYXI   2020-09-24   323517          34705556.0       NaN  
       2020-09-25   268316          34705556.0       NaN  
       2020-09-28   391262          34705556.0       NaN  
       2020-09-29   347361          34705556.0       NaN  
       2020-09-30  1014031          34737265.0       NaN  

[7104243 rows x 10 columns]

In [37]:
df_prices2.shape[0]

7104243

In [38]:
df_all=pd.concat([df_signals,df_prices2],axis=1)

In [39]:
df_all

(Dividends + Share Buyback) / FCF  Asset Turnover  \
Ticker Date                                                            
A      2007-01-03                                NaN             NaN   
       2007-01-04                                NaN             NaN   
       2007-01-05                                NaN             NaN   
       2007-01-08                                NaN             NaN   
       2007-01-09                                NaN             NaN   
...                                              ...             ...   
ZYXI   2020-09-24                            -0.0554        1.645039   
       2020-09-25                            -0.0554        1.645039   
       2020-09-28                            -0.0554        1.645039   
       2020-09-29                            -0.0554        1.645039   
       2020-09-30                            -0.0554        1.645039   

                   CapEx / (Depr + Amor)  Current Ratio  Dividends / FCF  \
Ticker Date                                                                
A      2007-01-03                    NaN            NaN              NaN   
       2007-01-04                    NaN            NaN              NaN   
       2007-01-05                    NaN            NaN              NaN   
       2007-01-08                    NaN            NaN              NaN   
       2007-01-09                    NaN            NaN              NaN   
...                                  ...            ...              ...   
ZYXI   2020-09-24                    NaN       4.955648         0.000468   
       2020-09-25                    NaN       4.955648         0.000468   
       2020-09-28                    NaN       4.955648         0.000468   
       2020-09-29                    NaN       4.955648         0.000468   
       2020-09-30                    NaN       4.955648         0.000468   

                   Gross Profit Margin  Interest Coverage  Log Revenue  \
Ticker Date                                                              
A      2007-01-03                  NaN                NaN          NaN   
       2007-01-04                  NaN                NaN          NaN   
       2007-01-05                  NaN                NaN          NaN   
       2007-01-08                  NaN                NaN          NaN   
       2007-01-09                  NaN                NaN          NaN   
...                                ...                ...          ...   
ZYXI   2020-09-24             0.792558         912.071429      7.78154   
       2020-09-25             0.792558         912.071429      7.78154   
       2020-09-28             0.792558         912.071429      7.78154   
       2020-09-29             0.792558         912.071429      7.78154   
       2020-09-30             0.792558         912.071429      7.78154   

                   Net Profit Margin  Quick Ratio  ...  SimFinId   Open  \
Ticker Date                                        ...                    
A      2007-01-03                NaN          NaN  ...     45846  34.99   
       2007-01-04                NaN          NaN  ...     45846  34.30   
       2007-01-05                NaN          NaN  ...     45846  34.30   
       2007-01-08                NaN          NaN  ...     45846  33.98   
       2007-01-09                NaN          NaN  ...     45846  34.08   
...                              ...          ...  ...       ...    ...   
ZYXI   2020-09-24           0.180817     4.016611  ...    171401  16.47   
       2020-09-25           0.180817     4.016611  ...    171401  15.89   
       2020-09-28           0.180817     4.016611  ...    171401  16.92   
       2020-09-29           0.180817     4.016611  ...    171401  16.86   
       2020-09-30           0.180817     4.016611  ...    171401  17.11   

                     Low   High  Close  Adj. Close  Dividend   Volume  \
Ticker Date                                                             
A      2

In [40]:
# remove rows that have NaN in Close_1y and Close columns
df_all2=df_all.dropna(subset=['Close', 'Close_1y'])

In [41]:
df_all2

(Dividends + Share Buyback) / FCF  Asset Turnover  \
Ticker Date                                                            
A      2007-01-03                                NaN             NaN   
       2007-01-04                                NaN             NaN   
       2007-01-05                                NaN             NaN   
       2007-01-08                                NaN             NaN   
       2007-01-09                                NaN             NaN   
...                                              ...             ...   
ZYXI   2019-04-15                           0.346088         1.65794   
       2019-04-16                           0.346088         1.65794   
       2019-04-17                           0.346088         1.65794   
       2019-04-18                           0.346088         1.65794   
       2019-04-22                           0.346088         1.65794   

                   CapEx / (Depr + Amor)  Current Ratio  Dividends / FCF  \
Ticker Date                                                                
A      2007-01-03                    NaN            NaN              NaN   
       2007-01-04                    NaN            NaN              NaN   
       2007-01-05                    NaN            NaN              NaN   
       2007-01-08                    NaN            NaN              NaN   
       2007-01-09                    NaN            NaN              NaN   
...                                  ...            ...              ...   
ZYXI   2019-04-15                    NaN       2.051267             -0.0   
       2019-04-16                    NaN       2.051267             -0.0   
       2019-04-17                    NaN       2.051267             -0.0   
       2019-04-18                    NaN       2.051267             -0.0   
       2019-04-22                    NaN       2.051267             -0.0   

                   Gross Profit Margin  Interest Coverage  Log Revenue  \
Ticker Date                                                              
A      2007-01-03                  NaN                NaN          NaN   
       2007-01-04                  NaN                NaN          NaN   
       2007-01-05                  NaN                NaN          NaN   
       2007-01-08                  NaN                NaN          NaN   
       2007-01-09                  NaN                NaN          NaN   
...                                ...                ...          ...   
ZYXI   2019-04-15             0.810822          67.337662     7.504022   
       2019-04-16             0.810822          67.337662     7.504022   
       2019-04-17             0.810822          67.337662     7.504022   
       2019-04-18             0.810822          67.337662     7.504022   
       2019-04-22             0.810822          67.337662     7.504022   

                   Net Profit Margin  Quick Ratio  ...  SimFinId   Open  \
Ticker Date                                        ...                    
A      2007-01-03                NaN          NaN  ...     45846  34.99   
       2007-01-04                NaN          NaN  ...     45846  34.30   
       2007-01-05                NaN          NaN  ...     45846  34.30   
       2007-01-08                NaN          NaN  ...     45846  33.98   
       2007-01-09                NaN          NaN  ...     45846  34.08   
...                              ...          ...  ...       ...    ...   
ZYXI   2019-04-15           0.299276     1.850064  ...    171401   5.20   
       2019-04-16           0.299276     1.850064  ...    171401   5.28   
       2019-04-17           0.299276     1.850064  ...    171401   5.31   
       2019-04-18           0.299276     1.850064  ...    171401   5.31   
       2019-04-22           0.299276     1.850064  ...    171401   5.30   

                     Low   High  Close  Adj. Close  Dividend   Volume  \
Ticker Date                                                             
A      2

In [42]:
# check those rows with Close as NaN
df_all.loc[df_all['Close'].isnull()]

(Dividends + Share Buyback) / FCF  Asset Turnover  \
Ticker Date                                                            
ADTM   2016-06-02                                NaN             NaN   
       2016-06-06                                NaN             NaN   
       2016-06-07                                NaN             NaN   
       2016-06-08                                NaN             NaN   
       2016-06-09                                NaN             NaN   
...                                              ...             ...   
WPTH   2013-12-13                                0.0             0.0   
       2013-12-16                                0.0             0.0   
       2013-12-17                                0.0             0.0   
       2013-12-18                                0.0             0.0   
       2013-12-19                                0.0             0.0   

                   CapEx / (Depr + Amor)  Current Ratio  Dividends / FCF  \
Ticker Date                                                                
ADTM   2016-06-02                    NaN            NaN              NaN   
       2016-06-06                    NaN            NaN              NaN   
       2016-06-07                    NaN            NaN              NaN   
       2016-06-08                    NaN            NaN              NaN   
       2016-06-09                    NaN            NaN              NaN   
...                                  ...            ...              ...   
WPTH   2013-12-13                    NaN       0.015032              0.0   
       2013-12-16                    NaN       0.015032              0.0   
       2013-12-17                    NaN       0.015032              0.0   
       2013-12-18                    NaN       0.015032              0.0   
       2013-12-19                    NaN       0.015032              0.0   

                   Gross Profit Margin  Interest Coverage  Log Revenue  \
Ticker Date                                                              
ADTM   2016-06-02                  NaN                NaN          NaN   
       2016-06-06                  NaN                NaN          NaN   
       2016-06-07                  NaN                NaN          NaN   
       2016-06-08                  NaN                NaN          NaN   
       2016-06-09                  NaN                NaN          NaN   
...                                ...                ...          ...   
WPTH   2013-12-13                  NaN          -0.202062         -inf   
       2013-12-16                  NaN          -0.202062         -inf   
       2013-12-17                  NaN          -0.202062         -inf   
       2013-12-18                  NaN          -0.202062         -inf   
       2013-12-19                  NaN          -0.202062         -inf   

                   Net Profit Margin  Quick Ratio  ...  SimFinId  Open  Low  \
Ticker Date                                        ...                        
ADTM   2016-06-02                NaN          NaN  ...    664460  0.01  NaN   
       2016-06-06                NaN          NaN  ...    664460  0.01  NaN   
       2016-06-07                NaN          NaN  ...    664460   NaN  NaN   
       2016-06-08                NaN          NaN  ...    664460   NaN  NaN   
       2016-06-09                NaN          NaN  ...    664460   NaN  NaN   
...                              ...          ...  ...       ...   ...  ...   
WPTH   2013-12-13               -inf     0.000912  ...    687927   NaN  NaN   
       2013-12-16               -inf     0.000912  ...    687927   NaN  NaN   
       2013-12-17               -inf     0.000912  ...    687927   NaN  NaN   
       2013-12-18               -inf     0.000912  ...    687927   NaN  NaN   
       2013-12-19               -inf     0.000912  ...    687927   NaN  NaN   

                   High  Close  Adj. Close  Dividend    Volume  \
Ticker Date                         

In [43]:
# drop all rows where there are at least 20 non-NaN.
df_all3=df_all2.dropna(thresh=20)

In [44]:
df_all3

(Dividends + Share Buyback) / FCF  Asset Turnover  \
Ticker Date                                                            
A      2010-09-29                           0.171492        0.553297   
       2010-09-30                           0.171492        0.553297   
       2010-10-01                           0.171492        0.553297   
       2010-10-04                           0.171492        0.553297   
       2010-10-05                           0.171492        0.553297   
...                                              ...             ...   
ZYXI   2019-04-15                           0.346088        1.657940   
       2019-04-16                           0.346088        1.657940   
       2019-04-17                           0.346088        1.657940   
       2019-04-18                           0.346088        1.657940   
       2019-04-22                           0.346088        1.657940   

                   CapEx / (Depr + Amor)  Current Ratio  Dividends / FCF  \
Ticker Date                                                                
A      2010-09-29               0.622857       1.966061             -0.0   
       2010-09-30               0.622857       1.966061             -0.0   
       2010-10-01               0.622857       1.966061             -0.0   
       2010-10-04               0.622857       1.966061             -0.0   
       2010-10-05               0.622857       1.966061             -0.0   
...                                  ...            ...              ...   
ZYXI   2019-04-15                    NaN       2.051267             -0.0   
       2019-04-16                    NaN       2.051267             -0.0   
       2019-04-17                    NaN       2.051267             -0.0   
       2019-04-18                    NaN       2.051267             -0.0   
       2019-04-22                    NaN       2.051267             -0.0   

                   Gross Profit Margin  Interest Coverage  Log Revenue  \
Ticker Date                                                              
A      2010-09-29             0.542205           5.636364     9.701999   
       2010-09-30             0.542205           5.636364     9.701999   
       2010-10-01             0.542205           5.636364     9.701999   
       2010-10-04             0.542205           5.636364     9.701999   
       2010-10-05             0.542205           5.636364     9.701999   
...                                ...                ...          ...   
ZYXI   2019-04-15             0.810822          67.337662     7.504022   
       2019-04-16             0.810822          67.337662     7.504022   
       2019-04-17             0.810822          67.337662     7.504022   
       2019-04-18             0.810822          67.337662     7.504022   
       2019-04-22             0.810822          67.337662     7.504022   

                   Net Profit Margin  Quick Ratio  ...  SimFinId   Open  \
Ticker Date                                        ...                    
A      2010-09-29           0.082820     1.065135  ...     45846  32.10   
       2010-09-30           0.082820     1.065135  ...     45846  32.77   
       2010-10-01           0.082820     1.065135  ...     45846  33.69   
       2010-10-04           0.082820     1.065135  ...     45846  33.56   
       2010-10-05           0.082820     1.065135  ...     45846  32.95   
...                              ...          ...  ...       ...    ...   
ZYXI   2019-04-15           0.299276     1.850064  ...    171401   5.20   
       2019-04-16           0.299276     1.850064  ...    171401   5.28   
       2019-04-17           0.299276     1.850064  ...    171401   5.31   
       2019-04-18           0.299276     1.850064  ...    171401   5.31   
       2019-04-22           0.299276     1.850064  ...    171401   5.30   

                     Low   High  Close  Adj. Close  Dividend   Volume  \
Ticker Date                                                             
A      2

## highest 10 price average next year

In [45]:
df_signals.index

MultiIndex([(   'A', '2007-01-03'),
            (   'A', '2007-01-04'),
            (   'A', '2007-01-05'),
            (   'A', '2007-01-08'),
            (   'A', '2007-01-09'),
            (   'A', '2007-01-10'),
            (   'A', '2007-01-11'),
            (   'A', '2007-01-12'),
            (   'A', '2007-01-16'),
            (   'A', '2007-01-17'),
            ...
            ('ZYXI', '2020-09-17'),
            ('ZYXI', '2020-09-18'),
            ('ZYXI', '2020-09-21'),
            ('ZYXI', '2020-09-22'),
            ('ZYXI', '2020-09-23'),
            ('ZYXI', '2020-09-24'),
            ('ZYXI', '2020-09-25'),
            ('ZYXI', '2020-09-28'),
            ('ZYXI', '2020-09-29'),
            ('ZYXI', '2020-09-30')],
           names=['Ticker', 'Date'], length=7104243)

In [46]:
# df_signals.index
# df_signals['Date'] = pd.to_datetime(df_signals['Date'],format='%Y%m%d')
# df_signals['Year'] = pd.DatetimeIndex(df_signals['Date']).year
# df_signals.groupby(by=["Ticker","Year"], dropna=False).max()

In [47]:
# price_1y

## Combine Signals & Returns

We then combine the signals and returns into a single DataFrame to align the data.

In [48]:
# %%time
# dfs = [df_signals, df_returns_1_3y]
# df_sig_rets = pd.concat(dfs, axis=1)

In [49]:
# %%time
# dfs = [df_signals, df_returns_1_1y]
# df_sig_rets = pd.concat(dfs, axis=1)

In [50]:
# dfs

In [51]:
# print all columns
# with pd.option_context('display.max_rows', 10, 'display.max_columns', None): 
#     display(df_sig_rets)

In [52]:
# print all columns
with pd.option_context('display.max_rows', 10, 'display.max_columns', None): 
    display(df_all3)

(Dividends + Share Buyback) / FCF  Asset Turnover  \
Ticker Date                                                            
A      2010-09-29                           0.171492        0.553297   
       2010-09-30                           0.171492        0.553297   
       2010-10-01                           0.171492        0.553297   
       2010-10-04                           0.171492        0.553297   
       2010-10-05                           0.171492        0.553297   
...                                              ...             ...   
ZYXI   2019-04-15                           0.346088        1.657940   
       2019-04-16                           0.346088        1.657940   
       2019-04-17                           0.346088        1.657940   
       2019-04-18                           0.346088        1.657940   
       2019-04-22                           0.346088        1.657940   

                   CapEx / (Depr + Amor)  Current Ratio  Dividends / FCF  \
Ticker Date                                                                
A      2010-09-29               0.622857       1.966061             -0.0   
       2010-09-30               0.622857       1.966061             -0.0   
       2010-10-01               0.622857       1.966061             -0.0   
       2010-10-04               0.622857       1.966061             -0.0   
       2010-10-05               0.622857       1.966061             -0.0   
...                                  ...            ...              ...   
ZYXI   2019-04-15                    NaN       2.051267             -0.0   
       2019-04-16                    NaN       2.051267             -0.0   
       2019-04-17                    NaN       2.051267             -0.0   
       2019-04-18                    NaN       2.051267             -0.0   
       2019-04-22                    NaN       2.051267             -0.0   

                   Gross Profit Margin  Interest Coverage  Log Revenue  \
Ticker Date                                                              
A      2010-09-29             0.542205           5.636364     9.701999   
       2010-09-30             0.542205           5.636364     9.701999   
       2010-10-01             0.542205           5.636364     9.701999   
       2010-10-04             0.542205           5.636364     9.701999   
       2010-10-05             0.542205           5.636364     9.701999   
...                                ...                ...          ...   
ZYXI   2019-04-15             0.810822          67.337662     7.504022   
       2019-04-16             0.810822          67.337662     7.504022   
       2019-04-17             0.810822          67.337662     7.504022   
       2019-04-18             0.810822          67.337662     7.504022   
       2019-04-22             0.810822          67.337662     7.504022   

                   Net Profit Margin  Quick Ratio  Return on Assets  \
Ticker Date                                                           
A      2010-09-29           0.082820     1.065135          0.045824   
       2010-09-30           0.082820     1.065135          0.045824   
       2010-10-01           0.082820     1.065135          0.045824   
       2010-10-04           0.082820     1.065135          0.045824   
       2010-10-05           0.082820     1.065135          0.045824   
...                              ...          ...               ...   
ZYXI   2019-04-15           0.299276     1.850064          0.496182   
       2019-04-16           0.299276     1.850064          0.496182   
       2019-04-17           0.299276     1.850064          0.496182   
       2019-04-18           0.299276     1.850064          0.496182   
       2019-04-22           0.299276     1.850064          0.496182   

                   Return on Equity  Share Buyback / FCF  Assets Growth  \
Ticker Date                                                               
A      2010-09-29          0.148399             0.171492

## Data Cleaning

The data also contains outliers and still has missing values, so let us fix that now. A common method for removing outliers is so-called *Winsorization* of the data. It basically just limits or clips the data between e.g. the 5% and 95% quantiles of the data. We will Winsorize both the stock-returns and all the signals. Then we remove all rows with missing values, and finally we remove tickers that have less than 200 rows of data.

There are more than 5 million data-rows before this cleaning:

In [53]:
# df_sig_rets.shape

In [54]:
df_all3.shape
# df_sig_rets['Total Return 1-3 Years']
# print(df_sig_rets.columns.tolist())

(3413239, 45)

In [55]:
print(df_all3.columns.tolist())

['(Dividends + Share Buyback) / FCF', 'Asset Turnover', 'CapEx / (Depr + Amor)', 'Current Ratio', 'Dividends / FCF', 'Gross Profit Margin', 'Interest Coverage', 'Log Revenue', 'Net Profit Margin', 'Quick Ratio', 'Return on Assets', 'Return on Equity', 'Share Buyback / FCF', 'Assets Growth', 'Assets Growth QOQ', 'Assets Growth YOY', 'Earnings Growth', 'Earnings Growth QOQ', 'Earnings Growth YOY', 'FCF Growth', 'FCF Growth QOQ', 'FCF Growth YOY', 'Sales Growth', 'Sales Growth QOQ', 'Sales Growth YOY', 'Earnings Yield', 'FCF Yield', 'Market-Cap', 'P/Cash', 'P/E', 'P/FCF', 'P/NCAV', 'P/NetNet', 'P/Sales', 'Price to Book Value', 'SimFinId', 'Open', 'Low', 'High', 'Close', 'Adj. Close', 'Dividend', 'Volume', 'Shares Outstanding', 'Close_1y']


## Remove all columns which don't have sufficient data.

In [56]:
# List of the columns before removing any.
columns_before = df_all3.columns

# Threshold for the number of rows that must be NaN for each column.
thresh = 0.75 * len(df_all3.dropna(how='all'))

# Remove all columns which don't have sufficient data.
df_all4 = df_all3.dropna(axis='columns', thresh=thresh)

# List of the columns after the removal.
columns_after = df_all4.columns

# Show the columns that were removed.
columns_before.difference(columns_after)

Index(['Dividend'], dtype='object')

In [57]:
df_all4.describe()

,(Dividends + Share Buyback) / FCF,Asset Turnover,CapEx / (Depr + Amor),Current Ratio,Dividends / FCF,Gross Profit Margin,Interest Coverage,Log Revenue,Net Profit Margin,Quick Ratio,...,Price to Book Value,SimFinId,Open,Low,High,Close,Adj. Close,Volume,Shares Outstanding,Close_1y
count,3.413239e+06,3.380839e+06,3.315935e+06,3.413239e+06,3.413239e+06,3.132578e+06,3.098523e+06,3.380523e+06,3.380839e+06,3.409261e+06,...,3.391077e+06,3.413239e+06,3.413238e+06,3.413238e+06,3.413239e+06,3.413239e+06,3.413239e+06,3.413239e+06,3.339415e+06,3.413239e+06
mean,NaN,1.821343e+01,inf,inf,inf,NaN,NaN,-inf,NaN,inf,...,inf,4.769466e+05,2.371585e+02,2.333634e+02,2.405062e+02,2.367931e+02,2.325861e+02,1.916914e+06,2.442006e+08,2.959198e+02
std,NaN,3.839664e+03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3.244512e+05,6.194812e+03,6.108963e+03,6.268181e+03,6.185600e+03,6.185682e+03,6.834471e+06,1.011791e+09,7.379684e+03
min,-inf,-8.229630e+01,-1.648176e+03,0.000000e+00,-3.641357e+03,-inf,-inf,-inf,-inf,0.000000e+00,...,-6.275763e+04,1.800000e+01,1.000000e-02,1.000000e-02,1.000000e-02,1.000000e-02,1.000000e-02,0.000000e+00,1.200000e+01,2.000000e-02
25%,-0.000000e+00,4.277727e-01,4.912143e-01,1.219251e+00,0.000000e+00,2.539976e-01,6.281709e-01,8.487547e+00,7.640423e-04,6.495053e-01,...,1.429805e+00,1.853500e+05,1.467000e+01,1.440000e+01,1.493000e+01,1.466000e+01,1.298000e+01,1.415150e+05,2.840588e+07,2.120000e+01
50%,2.738658e-01,7.561218e-01,8.855466e-01,1.876439e+00,-0.000000e+00,3.942652e-01,4.386179e+00,9.157210e+00,5.310483e-02,1.147869e+00,...,2.535781e+00,3.782250e+05,3.152000e+01,3.108000e+01,3.195000e+01,3.153000e+01,2.732000e+01,5.044000e+05,6.650446e+07,4.335000e+01
75%,8.631449e-01,1.267297e+00,1.450570e+00,3.068182e+00,2.910304e-01,5.949053e-01,1.365242e+01,9.678656e+00,1.131351e-01,2.062887e+00,...,4.616431e+00,7.371520e+05,5.800000e+01,5.728000e+01,5.870000e+01,5.800000e+01,5.173000e+01,1.594000e+06,1.863927e+08,7.854000e+01
max,inf,8.821851e+05,inf,inf,inf,inf,inf,1.171131e+01,inf,inf,...,inf,1.248288e+06,5.768438e+05,5.315625e+05,6.220594e+05,5.775000e+05,5.775000e+05,2.281025e+09,1.220330e+11,5.775000e+05


In [58]:
sf.winsorize?

In [59]:
%%time

# Clip both the signals and returns at their 5% and 95% quantiles.
# We do not set them to NaN because it would remove too much data.
df_all5 = sf.winsorize(df_all4,quantile=0.01)

# Remove all rows with missing values (NaN)
# because TensorFlow cannot handle that.
df_all5 = df_all5.dropna(how='any')

# Remove all tickers which have less than 200 data-rows.
df_all5 = df_all5.groupby(TICKER) \
                .filter(lambda df: len(df)>200)

CPU times: user 5.29 s, sys: 1.64 s, total: 6.93 s
Wall time: 6.98 s


This data-cleaning has removed about 80% of the rows, which is a lot:

In [60]:
df_all5.shape

(2294576, 44)

In [61]:
# make sure it's all numeric
df_all5.describe()

,(Dividends + Share Buyback) / FCF,Asset Turnover,CapEx / (Depr + Amor),Current Ratio,Dividends / FCF,Gross Profit Margin,Interest Coverage,Log Revenue,Net Profit Margin,Quick Ratio,...,Price to Book Value,SimFinId,Open,Low,High,Close,Adj. Close,Volume,Shares Outstanding,Close_1y
count,2.294576e+06,2.294576e+06,2.294576e+06,2.294576e+06,2.294576e+06,2.294576e+06,2.294576e+06,2.294576e+06,2.294576e+06,2.294576e+06,...,2.294576e+06,2.294576e+06,2.294576e+06,2.294576e+06,2.294576e+06,2.294576e+06,2.294576e+06,2.294576e+06,2.294576e+06,2.294576e+06
mean,4.763061e-01,9.961129e-01,1.123732e+00,2.374445e+00,1.942689e-01,4.219964e-01,9.127064e-01,9.166059e+00,-1.146311e-01,1.575641e+00,...,4.076694e+00,4.605828e+05,4.997692e+01,4.935979e+01,5.057951e+01,4.997922e+01,4.563606e+01,1.700316e+06,2.109778e+08,6.812122e+01
std,2.038256e+00,7.512247e-01,1.157652e+00,2.120184e+00,7.473673e-01,2.229721e-01,2.007161e+02,8.958856e-01,1.565991e+00,1.841977e+00,...,8.008460e+00,3.227287e+05,5.586995e+01,5.518991e+01,5.653438e+01,5.586441e+01,5.306167e+01,3.238598e+06,4.092862e+08,7.659317e+01
min,-9.802939e+00,3.335480e-03,-1.001267e+00,7.647490e-02,-3.174776e+00,-4.976890e-03,-1.038569e+03,6.018700e+00,-2.345730e+01,3.294064e-02,...,-2.669486e+01,3.762100e+04,5.500000e-01,5.300000e-01,5.700000e-01,5.500000e-01,5.100000e-01,0.000000e+00,8.947610e+05,1.040000e+00
25%,0.000000e+00,4.689325e-01,5.039351e-01,1.204613e+00,-0.000000e+00,2.510332e-01,9.066536e-01,8.704272e+00,1.034867e-02,6.242397e-01,...,1.510206e+00,1.822960e+05,1.666000e+01,1.639000e+01,1.693000e+01,1.666000e+01,1.496000e+01,1.675115e+05,3.264325e+07,2.373000e+01
50%,3.164154e-01,7.960127e-01,8.769531e-01,1.793390e+00,-0.000000e+00,3.878995e-01,4.751724e+00,9.270841e+00,5.594406e-02,1.076660e+00,...,2.603539e+00,3.781630e+05,3.542000e+01,3.497000e+01,3.588000e+01,3.542000e+01,3.130000e+01,5.709345e+05,7.547170e+07,4.810000e+01
75%,8.872133e-01,1.323867e+00,1.390040e+00,2.791534e+00,3.239832e-01,5.850011e-01,1.360325e+01,9.765296e+00,1.091120e-01,1.796394e+00,...,4.655949e+00,7.083980e+05,6.355000e+01,6.283000e+01,6.425000e+01,6.355000e+01,5.693000e+01,1.725300e+06,2.089176e+08,8.413000e+01
max,1.118182e+01,4.012165e+00,9.207011e+00,1.703641e+01,4.117004e+00,9.545284e-01,9.879403e+02,1.106425e+01,5.157104e-01,1.599324e+01,...,5.129521e+01,1.198893e+06,4.212263e+02,4.156000e+02,4.269900e+02,4.212500e+02,4.004462e+02,2.307782e+07,3.062390e+09,5.927100e+02


In [62]:
# make sure previous winsorize function didn't distort data too much.
df_all5.loc[[('NFLX', '2019-04-18'), ('NFLX', '2019-04-22')]]

(Dividends + Share Buyback) / FCF  Asset Turnover  \
Ticker Date                                                            
NFLX   2019-04-18                           0.043617        0.608073   
       2019-04-22                           0.043617        0.608073   

                   CapEx / (Depr + Amor)  Current Ratio  Dividends / FCF  \
Ticker Date                                                                
NFLX   2019-04-18               2.091778        1.49432              0.0   
       2019-04-22               2.091778        1.49432              0.0   

                   Gross Profit Margin  Interest Coverage  Log Revenue  \
Ticker Date                                                              
NFLX   2019-04-18             0.368917           3.817486    10.198502   
       2019-04-22             0.368917           3.817486    10.198502   

                   Net Profit Margin  Quick Ratio  ...  Price to Book Value  \
Ticker Date                                        ...                        
NFLX   2019-04-18           0.076688     0.584908  ...            31.038952   
       2019-04-22           0.076688     0.584908  ...            32.502395   

                   SimFinId   Open     Low    High   Close  Adj. Close  \
Ticker Date                                                              
NFLX   2019-04-18     83548  355.0  351.64  360.41  360.35      360.35   
       2019-04-22     83548  359.7  359.00  377.69  377.34      377.34   

                       Volume  Shares Outstanding  Close_1y  
Ticker Date                                                  
NFLX   2019-04-18   8353165.0         437191891.0    556.55  
       2019-04-22  11980534.0         437191891.0    556.55  

[2 rows x 44 columns]

In [63]:
# print all columns
with pd.option_context('display.max_rows', 10, 'display.max_columns', None): 
    display(df_all5)

(Dividends + Share Buyback) / FCF  Asset Turnover  \
Ticker Date                                                            
A      2011-09-29                           0.185950        0.738375   
       2011-09-30                           0.185950        0.738375   
       2011-10-03                           0.185950        0.738375   
       2011-10-04                           0.185950        0.738375   
       2011-10-05                           0.185950        0.738375   
...                                              ...             ...   
ZUMZ   2019-04-15                          -0.020258        1.831964   
       2019-04-16                          -0.020258        1.831964   
       2019-04-17                          -0.020258        1.831964   
       2019-04-18                          -0.020258        1.831964   
       2019-04-22                          -0.020258        1.831964   

                   CapEx / (Depr + Amor)  Current Ratio  Dividends / FCF  \
Ticker Date                                                                
A      2011-09-29               0.603113       3.470432             -0.0   
       2011-09-30               0.603113       3.470432             -0.0   
       2011-10-03               0.603113       3.470432             -0.0   
       2011-10-04               0.603113       3.470432             -0.0   
       2011-10-05               0.603113       3.470432             -0.0   
...                                  ...            ...              ...   
ZUMZ   2019-04-15               0.769805       3.513930             -0.0   
       2019-04-16               0.769805       3.513930             -0.0   
       2019-04-17               0.769805       3.513930             -0.0   
       2019-04-18               0.769805       3.513930             -0.0   
       2019-04-22               0.769805       3.513930             -0.0   

                   Gross Profit Margin  Interest Coverage  Log Revenue  \
Ticker Date                                                              
A      2011-09-29             0.532570          13.927536     9.810434   
       2011-09-30             0.532570          13.927536     9.810434   
       2011-10-03             0.532570          13.927536     9.810434   
       2011-10-04             0.532570          13.927536     9.810434   
       2011-10-05             0.532570          13.927536     9.810434   
...                                ...                ...          ...   
ZUMZ   2019-04-15             0.343276         -36.098109     8.990613   
       2019-04-16             0.343276         -36.098109     8.990613   
       2019-04-17             0.343276         -36.098109     8.990613   
       2019-04-18             0.343276         -36.098109     8.990613   
       2019-04-22             0.343276         -36.098109     8.990613   

                   Net Profit Margin  Quick Ratio  Return on Assets  \
Ticker Date                                                           
A      2011-09-29           0.157048     2.669767          0.115960   
       2011-09-30           0.157048     2.669767          0.115960   
       2011-10-03           0.157048     2.669767          0.115960   
       2011-10-04           0.157048     2.669767          0.115960   
       2011-10-05           0.157048     2.669767          0.115960   
...                              ...          ...               ...   
ZUMZ   2019-04-15           0.046193     1.966641          0.084623   
       2019-04-16           0.046193     1.966641          0.084623   
       2019-04-17           0.046193     1.966641          0.084623   
       2019-04-18           0.046193     1.966641          0.084623   
       2019-04-22           0.046193     1.966641          0.084623   

                   Return on Equity  Share Buyback / FCF  Assets Growth  \
Ticker Date                                                               
A      2011-09-29          0.241667             0.185950

In [64]:
# df_all5.index

In [65]:
# filter by one of the index
x=df_all5[df_all5.index.get_level_values('Ticker').isin(['IBM'])].sample(10)
with pd.option_context('display.max_rows', 10, 'display.max_columns', None): 
    display(x)

(Dividends + Share Buyback) / FCF  Asset Turnover  \
Ticker Date                                                            
IBM    2011-12-22                           0.955682        0.966330   
       2017-06-27                           0.727251        0.675697   
       2018-09-17                           0.690334        0.664101   
       2017-12-22                           0.812851        0.644275   
       2015-09-17                           0.577222        0.771015   
       2011-03-03                           0.915954        0.880309   
       2013-11-19                           0.988430        0.874932   
       2014-07-28                           1.688563        0.792484   
       2012-05-18                           0.982810        0.918262   
       2015-02-09                           1.634313        0.802101   

                   CapEx / (Depr + Amor)  Current Ratio  Dividends / FCF  \
Ticker Date                                                                
IBM    2011-12-22               0.729535       1.259354         0.212589   
       2017-06-27               0.672801       1.175626         0.430117   
       2018-09-17               0.680790       1.320363         0.427623   
       2017-12-22               0.601682       1.411477         0.449357   
       2015-09-17               0.802625       1.271563         0.321111   
       2011-03-03               0.706893       1.186233         0.196913   
       2013-11-19               0.670337       1.167532         0.262222   
       2014-07-28               0.736177       1.168051         0.307093   
       2012-05-18               0.726895       1.209002         0.212455   
       2015-02-09               0.733047       1.114856         0.301818   

                   Gross Profit Margin  Interest Coverage  Log Revenue  \
Ticker Date                                                              
IBM    2011-12-22             0.466345          49.984925    11.027142   
       2017-06-27             0.470859          18.360841    10.899771   
       2018-09-17             0.468293          19.093890    10.907250   
       2017-12-22             0.464979          17.773026    10.894133   
       2015-09-17             0.501409          35.304721    10.939095   
       2011-03-03             0.460734          49.323370    10.999439   
       2013-11-19             0.494010          47.063679    11.003249   
       2014-07-28             0.500376          45.973366    10.987644   
       2012-05-18             0.468948          49.357664    11.029043   
       2015-02-09             0.503151          40.827083    10.979444   

                   Net Profit Margin  Quick Ratio  Return on Assets  \
Ticker Date                                                           
IBM    2011-12-22           0.146793     0.583491          0.141851   
       2017-06-27           0.146263     0.522793          0.098830   
       2018-09-17           0.071227     0.540391          0.047302   
       2017-12-22           0.144321     0.620386          0.092982   
       2015-09-17           0.129816     0.489433          0.100090   
       2011-03-03           0.148532     0.554337          0.130754   
       2013-11-19           0.157974     0.502815          0.138216   
       2014-07-28           0.162910     0.472150          0.129103   
       2012-05-18           0.148294     0.548405          0.136173   
       2015-02-09           0.133365     0.441721          0.106973   

                   Return on Equity  Share Buyback / FCF  Assets Growth  \
Ticker Date                                                               
IBM    2011-12-22          0.698275             0.743093       0.027852   
       2017-06-27          0.629445             0.297134      -0.011467   
       2018-09-17          0.308505             0.262712       0.009378   
       2017-12-22          0.572455             0.363494       0.052169   
       2015-09-17          0.815481             0

In [66]:
with pd.option_context('display.max_rows', 10, 'display.max_columns', None): 
    display(df_all5.loc['A'].sample(10))

,(Dividends + Share Buyback) / FCF,Asset Turnover,CapEx / (Depr + Amor),Current Ratio,Dividends / FCF,Gross Profit Margin,Interest Coverage,Log Revenue,Net Profit Margin,Quick Ratio,Return on Assets,Return on Equity,Share Buyback / FCF,Assets Growth,Assets Growth QOQ,Assets Growth YOY,Earnings Growth,Earnings Growth QOQ,Earnings Growth YOY,FCF Growth,FCF Growth QOQ,FCF Growth YOY,Sales Growth,Sales Growth QOQ,Sales Growth YOY,Earnings Yield,FCF Yield,Market-Cap,P/Cash,P/E,P/FCF,P/NCAV,P/NetNet,P/Sales,Price to Book Value,SimFinId,Open,Low,High,Close,Adj. Close,Volume,Shares Outstanding,Close_1y
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-06-15,0.666165,0.538745,0.565957,3.337925,0.231579,0.526763,10.918033,9.627468,0.120019,2.657484,0.064660,0.118372,0.434586,0.078061,0.010008,0.078061,0.108932,0.333333,0.388430,0.206897,-0.538462,0.150685,0.049752,-0.039604,0.037938,0.026249,0.034294,1.939128e+10,8.652956,38.096807,29.159812,176.571262,-34.275342,4.572336,4.509599,45846,59.22,58.82,59.29,59.21,57.18,1091788.0,321337443.0,74.82
2015-02-10,0.348148,0.374295,0.497396,3.255910,0.325926,0.488142,4.148515,9.607241,0.135623,1.680851,0.050763,0.103507,0.022222,0.012072,0.041908,0.012072,-0.252044,-0.537415,-0.692308,-0.436913,-6.875000,-0.591304,0.039548,0.033697,-0.989426,0.040944,0.040273,1.340846e+10,6.045293,24.423424,24.830481,-167.434186,-5.286205,3.312367,2.527990,45846,39.31,38.78,39.76,39.67,37.41,1408747.0,335810297.0,47.01
2012-05-17,0.045863,0.739752,0.706349,3.351906,-0.000000,0.532907,14.881579,9.828080,0.155846,2.625220,0.115287,0.233163,0.045863,0.131154,0.004637,0.131154,0.314536,-0.204152,0.191710,0.632893,-0.774403,0.268293,0.170609,-0.053819,0.076366,0.074721,0.079208,1.403893e+10,3.833678,13.383153,12.624935,12.590966,99.566862,2.085712,3.120455,45846,41.90,39.62,41.91,39.63,26.24,4935800.0,594000000.0,52.99
2017-11-16,0.517293,0.531897,0.787037,3.219984,0.248120,0.534820,13.183333,9.642860,0.144060,2.611604,0.076625,0.137280,0.269173,0.068141,0.030564,0.068141,0.329832,0.067073,0.411290,-0.032023,-0.135514,0.088235,0.064954,0.010889,0.067050,0.028226,0.029653,2.242642e+10,8.750068,35.428791,33.723947,64.816257,-75.893147,5.103875,4.863679,45846,67.50,67.45,68.95,68.74,66.66,2268333.0,322000000.0,82.00
2015-05-05,0.439528,0.540763,0.500000,3.937500,0.489676,0.486719,4.659091,9.609167,0.102558,2.945043,0.055460,0.098349,-0.050147,-0.293194,-0.228035,-0.293194,-0.444000,-0.073529,-0.676923,-0.631922,-1.368794,-1.348993,0.261949,-0.016299,0.017857,0.029664,0.024115,1.405742e+10,6.637120,33.710839,41.467316,37.486453,-33.489982,3.457309,3.315429,45846,41.72,41.45,41.85,41.59,39.31,1254801.0,333191751.0,48.44
2017-07-19,0.673846,0.539421,0.671111,3.201348,0.246154,0.530990,12.065574,9.635886,0.134598,2.582982,0.072605,0.133029,0.427692,0.049215,0.018293,0.049215,0.257019,-0.023810,0.802198,0.072607,1.547619,-0.065502,0.055664,0.032802,0.081452,0.029019,0.032409,2.005592e+10,8.395109,34.460335,30.855254,126.137830,-42.649474,4.638278,4.584209,45846,61.32,61.17,61.64,61.38,59.41,1509141.0,321337443.0,74.82
2018-11-05,0.574627,0.575997,0.966019,3.616371,0.232587,0.545020,21.357143,9.682055,0.061967,2.824458,0.035693,0.065236,0.342040,0.010652,-0.049522,0.010652,-0.529226,0.151220,0.348571,0.209023,-0.356863,-0.113514,0.094447,-0.002488,0.079892,0.013912,0.037534,2.142075e+10,10.051971,71.881711,26.642724,-167.434186,-27.157845,4.454304,4.689306,45846,66.03,65.23,66.39,65.91,64.50,2348704.0,318000000.0,90.13
2015-01-20,0.348148,0.374295,0.497396,3.255910,0.325926,0.488142,4.148515,9.607241,0.135623,1.680851,0.050763,0.103507,0.022222,0.012072,0.041908,0.012072,-0.252044,-0.537415,-0.692308,-0.436913,-6.875000,-0.591304,0.039548,0.033697,-0.989426,0.042823,0.042121,1.282034e+10,5.780135,23.352168,23.741370,-167.434186,-5.054343,3.167080,2.417108,45846,38.43,37.76,38.66,37.93,35.77,5033505.0,335321802.0,46.68
2015-05-12,0.439528,0.540763,0.500000,3.937500,0.489676,0.486719,4.659091,9.609167,0.102558,2.945043,0.055

In [67]:
# remove some useless columns. This is important
df_all6=df_all5.drop(['SimFinId', 'Open', 'Low', 'High', 'Adj. Close'], axis=1)

In [68]:
df_all6.describe()

,(Dividends + Share Buyback) / FCF,Asset Turnover,CapEx / (Depr + Amor),Current Ratio,Dividends / FCF,Gross Profit Margin,Interest Coverage,Log Revenue,Net Profit Margin,Quick Ratio,...,P/E,P/FCF,P/NCAV,P/NetNet,P/Sales,Price to Book Value,Close,Volume,Shares Outstanding,Close_1y
count,2.294576e+06,2.294576e+06,2.294576e+06,2.294576e+06,2.294576e+06,2.294576e+06,2.294576e+06,2.294576e+06,2.294576e+06,2.294576e+06,...,2.294576e+06,2.294576e+06,2.294576e+06,2.294576e+06,2.294576e+06,2.294576e+06,2.294576e+06,2.294576e+06,2.294576e+06,2.294576e+06
mean,4.763061e-01,9.961129e-01,1.123732e+00,2.374445e+00,1.942689e-01,4.219964e-01,9.127064e-01,9.166059e+00,-1.146311e-01,1.575641e+00,...,1.733222e+01,1.512289e+01,-5.859821e-01,-1.664798e+00,3.218107e+00,4.076694e+00,4.997922e+01,1.700316e+06,2.109778e+08,6.812122e+01
std,2.038256e+00,7.512247e-01,1.157652e+00,2.120184e+00,7.473673e-01,2.229721e-01,2.007161e+02,8.958856e-01,1.565991e+00,1.841977e+00,...,6.571462e+01,7.155853e+01,3.360702e+01,3.210051e+01,8.413326e+00,8.008460e+00,5.586441e+01,3.238598e+06,4.092862e+08,7.659317e+01
min,-9.802939e+00,3.335480e-03,-1.001267e+00,7.647490e-02,-3.174776e+00,-4.976890e-03,-1.038569e+03,6.018700e+00,-2.345730e+01,3.294064e-02,...,-3.264351e+02,-3.565853e+02,-1.674342e+02,-1.793643e+02,7.919207e-02,-2.669486e+01,5.500000e-01,0.000000e+00,8.947610e+05,1.040000e+00
25%,0.000000e+00,4.689325e-01,5.039351e-01,1.204613e+00,-0.000000e+00,2.510332e-01,9.066536e-01,8.704272e+00,1.034867e-02,6.242397e-01,...,7.631445e+00,5.683040e+00,-4.945907e+00,-5.052182e+00,7.969743e-01,1.510206e+00,1.666000e+01,1.675115e+05,3.264325e+07,2.373000e+01
50%,3.164154e-01,7.960127e-01,8.769531e-01,1.793390e+00,-0.000000e+00,3.878995e-01,4.751724e+00,9.270841e+00,5.594406e-02,1.076660e+00,...,1.835374e+01,1.601526e+01,-1.453815e+00,-1.923843e+00,1.617801e+00,2.603539e+00,3.542000e+01,5.709345e+05,7.547170e+07,4.810000e+01
75%,8.872133e-01,1.323867e+00,1.390040e+00,2.791534e+00,3.239832e-01,5.850011e-01,1.360325e+01,9.765296e+00,1.091120e-01,1.796394e+00,...,2.812221e+01,2.693450e+01,3.971163e+00,-4.021269e-01,3.152076e+00,4.655949e+00,6.355000e+01,1.725300e+06,2.089176e+08,8.413000e+01
max,1.118182e+01,4.012165e+00,9.207011e+00,1.703641e+01,4.117004e+00,9.545284e-01,9.879403e+02,1.106425e+01,5.157104e-01,1.599324e+01,...,3.495670e+02,3.660790e+02,1.765713e+02,1.633674e+02,1.324815e+02,5.129521e+01,4.212500e+02,2.307782e+07,3.062390e+09,5.927100e+02


The reason that so many rows have been removed, is that TensorFlow cannot handle rows with missing values (NaN). Some missing values could be set to a default value, e.g. the Dividend Yield could be set to 0. Other columns could be omitted because they don't have any predictive power anyway.

## Split Training / Test-Sets

When training a Machine Learning model, we typically split the dataset into training- and test-sets. Only the training-set is used to train the model, after which the model is used on the test-set, to assess how the model performs on data it has not seen during training.

If we randomly divide all the data-rows into training- and test-sets, we would most likely get data for every stock-ticker in both the training- and test-sets, which would probably make them highly correlated. To avoid this, we split the dataset according to stock-tickers, so a ticker belongs to either the training- or test-set, but not both. We use 80% of all the tickers in the training-set, and 20% in the test-set.

In [69]:
# List of all unique stock-tickers in the dataset.
tickers = df_all6.reset_index()[TICKER].unique()

# Split the tickers into training- and test-sets.
tickers_train, tickers_test = \
    train_test_split(tickers, train_size=0.8, random_state=1234)

In [70]:
tickers_train, tickers_test

(array(['FANG', 'PAHC', 'ADI', ..., 'TDC', 'JAN', 'LOGI'], dtype=object),
 array(['SCHN', 'WWE', 'NHC', 'SPXC', 'BMS', 'LECO', 'SOFO', 'SVT', 'CBRE',
        'LFUS', 'PRXL', 'VOXX', 'HCSG', 'CALM', 'COST', 'NJR', 'PEAK',
        'RGEN', 'FCN', 'DENN', 'CASY', 'DEA', 'FIS', 'MCO', 'CNDT', 'LUNA',
        'EIX', 'DRI', 'EXC', 'STX', 'CWEN', 'SEV', 'KMX', 'ASCMA', 'UFI',
        'CP', 'STZ', 'HRL', 'ISSC', 'BWLA', 'BHR', 'ADSK', 'GIDYL', 'NXGN',
        'BLFS', 'SBAC', 'NEOS', 'JCI', 'EFX', 'SEIC', 'WOR', 'GIMO', 'AMD',
        'FBHS', 'MMM', 'ITT', 'TESO', 'PG', 'GPS', 'EHC', 'CYCC', 'TA',
        'UTHR', 'LE', 'VLRX', 'HON', 'HSIC', 'GMED', 'EPD', 'AROC', 'D',
        'TESS', 'SCI', 'SCVL', 'DE', 'R', 'SLP', 'LOW', 'IPHI', 'PLD',
        'FLXS', 'JSDA', 'COO', 'ENPH', 'COP', 'DESTQ', 'LSI', 'BRCD',
        'CELG', 'PBI', 'MVIS', 'HNI', 'SBH', 'CRD-A', 'J', 'CR', 'CUBE',
        'MCFT', 'IEP', 'SATS', 'FSS', 'RUTH', 'CFMS', 'CBM', 'HL', 'CZR',
        'COT', 'AXTA', 'MYCC', 'JNJ', 'KMG',

Now that we have lists of tickers for the training- and test-sets, we can select those rows from the DataFrame with signals and stock-returns:

In [71]:
%%time
df_train = df_all6.loc[tickers_train]
df_test = df_all6.loc[tickers_test]

CPU times: user 148 ms, sys: 83.9 ms, total: 231 ms
Wall time: 231 ms


We can then split these DataFrames into signals X and stock-returns y, that the Machine Learning model must try and find a mapping between. We use a common notation where the capital X indicates a 2-dimensional array or matrix, and the lower-case y indicates a 1-dimensional array.

In [72]:
# # DataFrames with signals for training- and test-sets.
# X_train = df_train.drop(columns=[TOTAL_RETURN_1_3Y])
# X_test = df_test.drop(columns=[TOTAL_RETURN_1_3Y])

# # DataFrames with returns for training- and test-sets.
# y_train = df_train[TOTAL_RETURN_1_3Y]
# y_test = df_test[TOTAL_RETURN_1_3Y]
df_train

(Dividends + Share Buyback) / FCF  Asset Turnover  \
Ticker Date                                                            
FANG   2018-03-01                           0.196157        0.155064   
       2018-03-02                           0.196157        0.155064   
       2018-03-05                           0.196157        0.155064   
       2018-03-06                           0.196157        0.155064   
       2018-03-07                           0.196157        0.155064   
...                                              ...             ...   
LOGI   2019-04-15                           0.454074        1.272728   
       2019-04-16                           0.454074        1.272728   
       2019-04-17                           0.454074        1.272728   
       2019-04-18                           0.454074        1.272728   
       2019-04-22                           0.454074        1.272728   

                   CapEx / (Depr + Amor)  Current Ratio  Dividends / FCF  \
Ticker Date                                                                
FANG   2018-03-01               8.503062       0.621032         0.000000   
       2018-03-02               8.503062       0.621032         0.000000   
       2018-03-05               8.503062       0.621032         0.000000   
       2018-03-06               8.503062       0.621032         0.000000   
       2018-03-07               8.503062       0.621032         0.000000   
...                                  ...            ...              ...   
LOGI   2019-04-15               0.613740       1.622817         0.352676   
       2019-04-16               0.613740       1.622817         0.352676   
       2019-04-17               0.613740       1.622817         0.352676   
       2019-04-18               0.613740       1.622817         0.352676   
       2019-04-22               0.613740       1.622817         0.352676   

                   Gross Profit Margin  Interest Coverage  Log Revenue  \
Ticker Date                                                              
FANG   2018-03-01             0.954528          14.780488     9.080987   
       2018-03-02             0.954528          14.780488     9.080987   
       2018-03-05             0.954528          14.780488     9.080987   
       2018-03-06             0.954528          14.780488     9.080987   
       2018-03-07             0.954528          14.780488     9.080987   
...                                ...                ...          ...   
LOGI   2019-04-15             0.368690         -37.299301     9.440349   
       2019-04-16             0.368690         -37.299301     9.440349   
       2019-04-17             0.368690         -37.299301     9.440349   
       2019-04-18             0.368690         -37.299301     9.440349   
       2019-04-22             0.368690         -37.299301     9.440349   

                   Net Profit Margin  Quick Ratio  ...        P/E      P/FCF  \
Ticker Date                                        ...                         
FANG   2018-03-01           0.400000     0.469359  ...  25.619793  -6.534323   
       2018-03-02           0.400000     0.469359  ...  26.160927  -6.672339   
       2018-03-05           0.400000     0.469359  ...  26.398054  -6.732818   
       2018-03-06           0.400000     0.469359  ...  25.757610  -6.569473   
       2018-03-07           0.400000     0.469359  ...  25.591419  -6.527086   
...                              ...          ...  ...        ...        ...   
LOGI   2019-04-15           0.090644     1.168741  ...  27.906067  21.575928   
       2019-04-16           0.090644     1.168741  ...  28.122603  21.743345   
       2019-04-17           0.090644     1.168741  ...  28.169970  21.779968   
       2019-04-18           0.090644     1.168741  ...  28.136137  21.753809   
       2019-04-22           0.090644     1.168741  ...  28.129370  21.748577   

                      P/NCAV   P/NetNet    P/Sales  Price to Book Value  \
Ticker Date   

In [73]:
df_train.describe()

,(Dividends + Share Buyback) / FCF,Asset Turnover,CapEx / (Depr + Amor),Current Ratio,Dividends / FCF,Gross Profit Margin,Interest Coverage,Log Revenue,Net Profit Margin,Quick Ratio,...,P/E,P/FCF,P/NCAV,P/NetNet,P/Sales,Price to Book Value,Close,Volume,Shares Outstanding,Close_1y
count,1.823353e+06,1.823353e+06,1.823353e+06,1.823353e+06,1.823353e+06,1.823353e+06,1.823353e+06,1.823353e+06,1.823353e+06,1.823353e+06,...,1.823353e+06,1.823353e+06,1.823353e+06,1.823353e+06,1.823353e+06,1.823353e+06,1.823353e+06,1.823353e+06,1.823353e+06,1.823353e+06
mean,4.789881e-01,9.886069e-01,1.118524e+00,2.382962e+00,2.016489e-01,4.236188e-01,-4.618480e-02,9.159288e+00,-7.147296e-02,1.585680e+00,...,1.783142e+01,1.512051e+01,-3.448961e-01,-1.661734e+00,3.138084e+00,4.113382e+00,5.007452e+01,1.691327e+06,2.050738e+08,6.820005e+01
std,2.062562e+00,7.463234e-01,1.159464e+00,2.170443e+00,7.593180e-01,2.246456e-01,1.998102e+02,8.910338e-01,1.239057e+00,1.897126e+00,...,6.625988e+01,7.212822e+01,3.293276e+01,3.250129e+01,7.449609e+00,8.092209e+00,5.646151e+01,3.266877e+06,3.958918e+08,7.775029e+01
min,-9.802939e+00,3.335480e-03,-1.001267e+00,7.647490e-02,-3.174776e+00,-4.976890e-03,-1.038569e+03,6.018700e+00,-2.345730e+01,3.294064e-02,...,-3.264351e+02,-3.565853e+02,-1.674342e+02,-1.793643e+02,7.919207e-02,-2.669486e+01,5.500000e-01,0.000000e+00,8.947610e+05,1.040000e+00
25%,0.000000e+00,4.675937e-01,4.980735e-01,1.189907e+00,0.000000e+00,2.506077e-01,1.119313e+00,8.692623e+00,1.068695e-02,6.180318e-01,...,8.070593e+00,5.976984e+00,-4.821261e+00,-4.975041e+00,8.014995e-01,1.519119e+00,1.685000e+01,1.599000e+05,3.138289e+07,2.385000e+01
50%,3.097582e-01,7.856800e-01,8.699165e-01,1.780258e+00,-0.000000e+00,3.870043e-01,4.798180e+00,9.257087e+00,5.654970e-02,1.070641e+00,...,1.837990e+01,1.593831e+01,-1.402079e+00,-1.902978e+00,1.628266e+00,2.560716e+00,3.529000e+01,5.607910e+05,7.452946e+07,4.800000e+01
75%,8.855499e-01,1.304779e+00,1.388698e+00,2.803244e+00,3.326572e-01,5.899956e-01,1.334846e+01,9.756180e+00,1.115494e-01,1.782412e+00,...,2.850666e+01,2.710007e+01,3.924192e+00,-4.510001e-01,3.150666e+00,4.643075e+00,6.349000e+01,1.709600e+06,2.101853e+08,8.424000e+01
max,1.118182e+01,4.012165e+00,9.207011e+00,1.703641e+01,4.117004e+00,9.545284e-01,9.879403e+02,1.106425e+01,5.157104e-01,1.599324e+01,...,3.495670e+02,3.660790e+02,1.765713e+02,1.633674e+02,1.324815e+02,5.129521e+01,4.212500e+02,2.307782e+07,3.062390e+09,5.927100e+02


In [74]:
# DataFrames with signals for training- and test-sets.
X_train = df_train.drop(columns=['Close_1y'])
X_test = df_test.drop(columns=['Close_1y'])

# DataFrames with returns for training- and test-sets.
y_train = df_train['Close_1y']
y_test = df_test['Close_1y']

Let us check that the shapes of these arrays look reasonable:

In [75]:
X_train.shape, y_train.shape

((1823353, 38), (1823353,))

In [76]:
X_test.shape, y_test.shape

((471223, 38), (471223,))

Let us also get the number of signals, which will be convenient later:

In [77]:
num_signals = X_train.shape[1]
num_signals

38

## Scale Signals

Neural Networks require that the input has "moderate" values e.g. between 0 and 1, or mean 0 and 1 standard deviation, which is the method we use here. It is not necessary to scale the stock-returns because they have already been Winsorized to remove outliers, and they are centered around 0 with a small standard deviation.

If you want to input new data to the Neural Network, then you have to run the signals through the scaler first, because the Neural Network has been tuned to the scaled signals.

In [78]:
# Create a scaler and fit it to the signals in the training-data.
signal_scaler = StandardScaler()
signal_scaler.fit(X_train)

def scale_signal(df):
    """Apply the signal-scaler to the given DataFrame."""

    # Apply the scaler. This returns a numpy array.
    array = signal_scaler.transform(df)
    
    # Create a DataFrame with the correct column-names and index.
    df_scaled = pd.DataFrame(data=array,
                             columns=df.columns,
                             index=df.index)
    return df_scaled

# Scale the signals in the training- and test-sets.
X_train = scale_signal(X_train)
X_test = scale_signal(X_test)

Except for small rounding errors, the signals in the training-set now have 0 mean and 1 standard deviation:

In [79]:
X_train.describe()

,(Dividends + Share Buyback) / FCF,Asset Turnover,CapEx / (Depr + Amor),Current Ratio,Dividends / FCF,Gross Profit Margin,Interest Coverage,Log Revenue,Net Profit Margin,Quick Ratio,...,P/Cash,P/E,P/FCF,P/NCAV,P/NetNet,P/Sales,Price to Book Value,Close,Volume,Shares Outstanding
count,1.823353e+06,1.823353e+06,1.823353e+06,1.823353e+06,1.823353e+06,1.823353e+06,1.823353e+06,1.823353e+06,1.823353e+06,1.823353e+06,...,1.823353e+06,1.823353e+06,1.823353e+06,1.823353e+06,1.823353e+06,1.823353e+06,1.823353e+06,1.823353e+06,1.823353e+06,1.823353e+06
mean,1.745812e-18,7.050586e-16,5.965689e-16,6.082908e-16,1.634828e-16,9.938658e-16,-3.697380e-17,1.555020e-16,1.247009e-19,5.200026e-17,...,1.560008e-16,1.134778e-17,6.704230e-17,9.259038e-18,4.214889e-17,-1.148495e-16,2.022648e-16,-5.182567e-16,8.754000e-17,-3.928077e-17
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-4.985028e+00,-1.320167e+00,-1.828251e+00,-1.062680e+00,-4.446656e+00,-1.907876e+00,-5.197548e+00,-3.524657e+00,-1.887389e+01,-8.184697e-01,...,-3.624857e-01,-5.195702e+00,-5.153405e+00,-5.073651e+00,-5.467555e+00,-4.106111e-01,-3.807150e+00,-8.771380e-01,-5.177198e-01,-5.157447e-01
25%,-2.322297e-01,-6.981066e-01,-5.351186e-01,-5.496826e-01,-2.655658e-01,-7.701519e-01,5.833025e-03,-5.237341e-01,6.630844e-02,-5.100604e-01,...,-3.237357e-01,-1.473113e-01,-1.267676e-01,-1.359244e-01,-1.019439e-01,-3.136520e-01,-3.205878e-01,-5.884456e-01,-4.687740e-01,-4.387334e-01
50%,-8.204840e-02,-2.719022e-01,-2.144161e-01,-2.776869e-01,-2.655658e-01,-1.629881e-01,2.424483e-02,1.097593e-01,1.033227e-01,-2.714843e-01,...,-2.696234e-01,8.277683e-03,1.133816e-02,-3.210125e-02,-7.422588e-03,-2.026708e-01,-1.918718e-01,-2.618514e-01,-3.460601e-01,-3.297476e-01
75%,1.971149e-01,4.236396e-01,2.330164e-01,1.936391e-01,1.725343e-01,7.406192e-01,6.703685e-02,6.698870e-01,1.477111e-01,1.036999e-01,...,-1.138798e-01,1.611118e-01,1.660872e-01,1.296305e-01,3.725189e-02,1.688936e-03,6.545727e-02,2.376041e-01,5.593518e-03,1.291126e-02
max,5.189095e+00,4.051271e+00,6.976056e+00,6.751365e+00,5.156412e+00,2.363322e+00,4.944625e+00,2.137922e+00,4.738955e-01,7.594418e+00,...,7.669713e+00,5.006584e+00,4.865760e+00,5.372043e+00,5.077619e+00,1.736245e+01,5.830526e+00,6.573958e+00,6.546467e+00,7.217419e+00


Using the same scaler on the test-data, also gives means that are close to 0 and standard deviations close to 1:

In [80]:
X_test.describe()

,(Dividends + Share Buyback) / FCF,Asset Turnover,CapEx / (Depr + Amor),Current Ratio,Dividends / FCF,Gross Profit Margin,Interest Coverage,Log Revenue,Net Profit Margin,Quick Ratio,...,P/Cash,P/E,P/FCF,P/NCAV,P/NetNet,P/Sales,Price to Book Value,Close,Volume,Shares Outstanding
count,471223.000000,471223.000000,471223.000000,471223.000000,471223.000000,471223.000000,471223.000000,471223.000000,471223.000000,471223.000000,...,471223.000000,471223.000000,471223.000000,471223.000000,471223.000000,471223.000000,471223.000000,471223.000000,471223.000000,471223.000000
mean,-0.006332,0.048973,0.021871,-0.019108,-0.047326,-0.035168,0.023368,0.037004,-0.169608,-0.025769,...,0.004350,-0.036686,0.000161,-0.035647,-0.000459,0.052307,-0.022077,-0.008219,0.013399,0.072618
std,0.941212,1.030662,0.992176,0.881367,0.919865,0.962674,1.021675,1.025713,1.971248,0.848813,...,1.111322,0.958708,0.960930,1.095639,0.938429,1.529438,0.948339,0.947366,0.957039,1.153640
min,-4.985028,-1.320167,-1.828251,-1.062680,-4.446656,-1.907876,-5.197548,-3.524657,-18.873894,-0.818470,...,-0.362486,-5.195702,-5.153405,-5.073651,-5.467555,-0.410611,-3.807150,-0.877138,-0.517720,-0.515745
25%,-0.232230,-0.684538,-0.507926,-0.517033,-0.265566,-0.760687,0.000670,-0.485188,0.064909,-0.490751,...,-0.325417,-0.189710,-0.151492,-0.153239,-0.115196,-0.316385,-0.327449,-0.606157,-0.456438,-0.418122
50%,-0.064239,-0.219304,-0.188257,-0.252403,-0.265566,-0.150746,0.023149,0.195143,0.101237,-0.256626,...,-0.284884,0.006064,0.016484,-0.041408,-0.010461,-0.209407,-0.164540,-0.249808,-0.331797,-0.320177
75%,0.202266,0.564597,0.241790,0.163834,0.123615,0.612916,0.073389,0.711379,0.138584,0.137946,...,-0.156093,0.136318,0.156734,0.138819,0.044758,0.002622,0.071459,0.243449,0.030071,-0.011203
max,5.189095,4.051271,6.976056,6.751365,5.156412,2.363322,4.944625,2.137922,0.473896,7.594418,...,7.669713,5.006584,4.865760,5.372043,5.077619,17.362451,5.830526,6.573958,6.546467,7.217419


In [81]:
y_train.values

array([139.92, 139.92, 139.92, ...,  75.84,  77.14,  77.31])

## Neural Network: Regression -- Grid Search

https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/

In [82]:
from sklearn.model_selection import GridSearchCV

In [88]:
from keras.wrappers.scikit_learn import KerasRegressor

In [ ]:
# https://medium.com/@gautam.karmakar/linear-regression-using-neural-network-d8815324017f

In [97]:
# Function to create model, required for KerasClassifier
def create_model(optimizer='adam'):
    model_regr = Sequential()

    activation = 'relu'

    # Add an input layer for the signals.
    # Note how we set the dimensionality of the input here.
    model_regr.add(InputLayer(input_shape=(num_signals,)))

    # Add several dense aka. fully-connected layers.
    # You can experiment with different designs.
    model_regr.add(Dense(128, activation=activation))
    model_regr.add(Dense(64, activation=activation))
    model_regr.add(Dense(32, activation=activation))
    model_regr.add(Dense(16, activation=activation))
    model_regr.add(Dense(8, activation=activation))

    # Add a layer for the output of the Neural Network.
    # This is 1-dimensional to match the stock-return data.
    model_regr.add(Dense(1))

    # Compile the model but don't train it yet.
    model_regr.compile(loss='mse', metrics=['mae'],
                       optimizer=RMSprop(0.001))
    
    return model_regr
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [98]:
# create model
model = KerasRegressor(build_fn=create_model, epochs=20, batch_size=10, verbose=0)

In [99]:
# Show the model.
# model_regr.summary()

In [ ]:
# define the grid search parameters
batch_size = [10, 100, 1000,10000]
epochs = [10, 50, 100]
weight_constraint = [1, 2, 3, 4, 5]
init = ['glorot_uniform', 'normal', 'uniform']
optimizers = ['rmsprop', 'adam']
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
param_grid = dict(batch_size=batch_size, epochs=epochs,dropout_rate=dropout_rate, optimizer=optimizers, weight_constraint=weight_constraint,init=init)
grid = GridSearchCV(estimator=model, param_grid=param_grid,n_jobs=3)
grid_result = grid.fit(X=X_train.values, y=y_train.values)

2021-10-10 00:03:23.033567: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2021-10-10 00:03:23.033603: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: pavilion690
2021-10-10 00:03:23.033611: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: pavilion690
2021-10-10 00:03:23.033697: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 450.119.3
2021-10-10 00:03:23.033724: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 450.119.3
2021-10-10 00:03:23.033729: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 450.119.3
2021-10-10 00:03:23.034133: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-cr

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [101]:
GridSearchCV?

In [105]:
# summarize results
print(“Best: %f using %s” % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_[‘mean_test_score’]
stds = grid_result.cv_results_[‘std_test_score’]
params = grid_result.cv_results_[‘params’]
for mean, stdev, param in zip(means, stds, params):
    print(“%f (%f) with: %r” % (mean, stdev, param))

SyntaxError: invalid character '“' (U+201C) (3391613668.py, line 2)

## Neural Network: Regression

Let us now build a Neural Network to learn the mapping from signals X to stock-returns y. We will use Keras and TensorFlow. If you are new to this, you can watch [this tutorial](https://www.youtube.com/watch?v=3yfRJKA1BiQ) for a basic explanation.

In [ ]:
# This so-called "activation function" is applied after each layer.
# It is important to use a non-linear function, otherwise
# the Neural Network cannot learn non-linear mappings.

# activation = 'relu'
# activation = 'selu'
activation = 'softplus'
# activation = 'tanh'

In [ ]:
# Create a new Keras model.
model_regr = Sequential()

# Add an input layer for the signals.
# Note how we set the dimensionality of the input here.
model_regr.add(InputLayer(input_shape=(num_signals,)))

# Add several dense aka. fully-connected layers.
# You can experiment with different designs.
model_regr.add(Dense(128, activation=activation))
model_regr.add(Dense(64, activation=activation))
model_regr.add(Dense(32, activation=activation))
model_regr.add(Dense(16, activation=activation))
model_regr.add(Dense(8, activation=activation))

# Add a layer for the output of the Neural Network.
# This is 1-dimensional to match the stock-return data.
model_regr.add(Dense(1))

# Compile the model but don't train it yet.
model_regr.compile(loss='mse', metrics=['mae'],
                   optimizer=RMSprop(0.001))

# Show the model.
model_regr.summary()

In [ ]:
#add dropout, worse than without dropout?

from keras.layers import Dropout

# Create a new Keras model.
model_regr = Sequential()

# Add an input layer for the signals.
# Note how we set the dimensionality of the input here.
model_regr.add(InputLayer(input_shape=(num_signals,)))
# model_regr.add(Dropout(0.1))

# Add several dense aka. fully-connected layers.
# You can experiment with different designs.
model_regr.add(Dense(128, activation=activation))
# model_regr.add(Dropout(0.1))
model_regr.add(Dense(64, activation=activation))
# model_regr.add(Dropout(0.1))
model_regr.add(Dense(32, activation=activation))
# model_regr.add(Dropout(0.1))
model_regr.add(Dense(16, activation=activation))
# model_regr.add(Dropout(0.1))

model_regr.add(Dense(8, activation=activation))
# model_regr.add(Dropout(0.1))

# Add a layer for the output of the Neural Network.
# This is 1-dimensional to match the stock-return data.
model_regr.add(Dense(1))

# Compile the model but don't train it yet.
model_regr.compile(loss='mse', metrics=['mae'],
                   optimizer=RMSprop(0.001))

# Show the model.
model_regr.summary()

These are common arguments for training both our regression and classification models:

In [ ]:
fit_args = \
{
    # For efficiency, the model is trained on batches of data.
    'batch_size': 40960,
    # 40960 seems better than orignal 4096
    
    # Number of iterations aka. epochs over the training-set.
    'epochs': 20,
    
    # Fraction of the training-set used for validation after
    # each training-epoch, to assess how well the model performs
    # on unseen data.
    'validation_split': 0.2,

    # Show status during training.
    'verbose': 1,
}

We can now train the model. This takes a minute or two on an ordinary computer:

In [ ]:
%%time
history_regr = model_regr.fit(x=X_train.values,
                              y=y_train.values, **fit_args)

We can then plot the training-progress as the Mean Absolute Error (MAE) which is supposed to be minimized. After 40 epochs through the training-set, the model has learned to map from signals to stock-returns for the training-set, with an average absolute error of about 6%, which would be very good if it generalized to unseen data. Unfortunately, the performance on the validation-set gets worse during optimization and ends with a MAE about 24%, so the model is clearly not learning the underlying relation between signals and stock-returns, but it has instead learned peculiarities and noise in the training-data.

In [ ]:
# Convert training history to a DataFrame.
df = pd.DataFrame(history_regr.history)

# Rename the relevant columns.
names = {'mae': 'Training MAE', 'val_mae': 'Validation MAE'}
df = df[['mae', 'val_mae']].rename(columns=names)

# Plot the results.
df.plot()

### Regression Results

Let us now compute some performance measures for the Neural Network. First we use the fitted model to predict the stock-returns for the training- and test-sets:

In [ ]:
%%time
y_train_pred = model_regr.predict(X_train.values)
y_test_pred = model_regr.predict(X_test.values)

We can then calculate the $R^2$ value (aka. the coefficient of determination) between the actual stock-returns and those predicted by the model. A value of 1 means a perfect prediction, while a value of 0 means the model basically predicted the average stock-return, and a negative value means the model's prediction was worse than just using the average stock-return.

The $R^2$ value is about 0.5 for the training-set, which is OK but could be much higher if we trained it more, and if we used a bigger Neural Network with more parameters to optimize.

In [ ]:
r2_score(y_true=y_train, y_pred=y_train_pred)

But the $R^2$ value is very negative for the test-set, which is so bad, that it would be much better to just use the average stock-return. This shows the model has not learned the underlying relation between the signals X and stock-returns y. The model has merely learned noise or peculiarities in the training-data, which do not generalize to the unseen data in the test-set:

In [ ]:
r2_score(y_true=y_test, y_pred=y_test_pred)

### The results are very good actually.
Without dropout, the $R^2$ value for training and test set are: 0.94,0.88, which is very good.

With dropout 0.2 at each layer, the $R^2$ value for training and test set are: 0.87,0.83, which is not as good.


With dropout 0.1 at each layer, the $R^2$ value for training and test set are: 0.92,0.88.

activation = 'softplus' decrease mae, the $R^2$ value for training and test set are: 0.937,0.895.


### Regression Examples

We can also try and plot the model's predicted versus actual stock-returns for a stock in the training-set. As we can see, the model has learned this mapping reasonably well:

In [ ]:
# ticker="A"

In [ ]:
# Column-name for the models' predicted 1 year max price.
price_1y_PRED = 'max price 1year Predicted'

# Create a DataFrame with actual and predicted 1 year max price.
# This is for the training-set.
df_y_train = pd.DataFrame(y_train)
df_y_train[price_1y_PRED] = y_train_pred

# Plot the actual and predicted stock-returns for the first stock.
ticker = tickers_train[11]
_ = df_y_train.loc[ticker].plot(title=ticker)

Now let us try and plot the model's predicted versus actual stock-returns for the first stock in the test-set. As we can see, the model is mostly unable to predict the stock-returns, for this data that it has not seen during its training:

In [ ]:
# Create a DataFrame with actual and predicted stock-returns.
# This is for the test-set.
df_y_test = pd.DataFrame(y_test)
df_y_test[price_1y_PRED] = y_test_pred

# Plot the actual and predicted stock-returns for the first stock.
ticker = tickers_test[12]
_ = df_y_test.loc[ticker].plot(title=ticker)

In [ ]:
tickers_test

In [ ]:
onestock_index=np.where(tickers_test=='NFLX')

# Plot the actual and predicted stock-returns for the first stock.
ticker = tickers_test[onestock_index]
ticker=ticker[0]
# ticker = np.array_str(ticker)
ticker
_ = df_y_test.loc[ticker].plot(title=ticker)

## The reason it doesn't predict NFLX well is probably NFLX is a early growth stock
For such stock, price reflects more of the speculative information, such as news, R&D development, exponential sales forecast.

In [ ]:
NFLX_prediction=df_y_test.loc[ticker]
NFLX_prediction

In [ ]:
NFLX_prediction.to_csv('prediction/NFLX_regression_prediction.csv', index=True)

In [ ]:
with pd.option_context('display.max_rows', 10, 'display.max_columns', None): 
    display(df_all5.loc['NFLX'].sample(10))

In [ ]:
df_all6.loc['NFLX']

In [ ]:
NFLX_pred=pd.concat([df_all6.loc['NFLX'][['Close']] ,NFLX_prediction],axis=1)
NFLX_pred

In [ ]:
NFLX_pred.to_csv('prediction/NFLX_regression_prediction2.csv', index=True)

In [ ]:
df_prices2.loc['NFLX']
df_prices2[df_prices2.index.get_level_values('Ticker').isin(['NFLX'])]
df_prices2.loc[[('NFLX', '2019-04-18'), ('NFLX', '2019-04-22')]]

In [ ]:
# df_all6.loc[[('NFLX', '2019-04-18'), ('NFLX', '2019-04-22')]]

In [ ]:
# df_all4.loc[[('NFLX', '2019-04-18'), ('NFLX', '2019-04-22')]]

In [ ]:
# df_all5.loc[[('NFLX', '2019-04-18'), ('NFLX', '2019-04-22')]]

## Save model to file

In [ ]:
# https://stackoverflow.com/questions/67661244/how-to-pickle-weakref-in-python-tensorflow-keras?noredirect=1
filename = '/home/upsman/MLAI/simfin_my/models/year1maxPrice_model3.sav'
tf.keras.models.save_model(model_regr, filename)

In [ ]:
loaded_model = tf.keras.models.load_model('/home/upsman/MLAI/simfin_my/models/year1maxPrice_model3.sav')

In [ ]:
loaded_model.summary()

In [ ]:
# https://stackoverflow.com/questions/67661244/how-to-pickle-weakref-in-python-tensorflow-keras?noredirect=1
model_regr.save('/home/upsman/MLAI/simfin_my/models/year1maxPrice_model3.h5')

In [ ]:
from keras.models import load_model
loaded_model_h5 = load_model('/home/upsman/MLAI/simfin_my/models/year1maxPrice_model3.h5')
loaded_model_h5.summary()

In [ ]:
# https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/
import pickle
# save the model to disk
filename = 'year1maxPrice_model.sav'
pickle.dump(model_regr, open(filename, 'wb'))

In [ ]:
import dill
import weakref
# dill.dumps(weakref.WeakKeyDictionary())
dill.dump(model_regr, open('year1maxPrice_model.pkl', 'wb'))

In [ ]:
import joblib
filename = 'year1maxPrice_model.sav'
joblib.dump(model_regr, filename)

In [ ]:
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, Y_test)
print(result)

## Neural Network: Classification

Instead of mapping from signals X to a continuous variable y for the stock-return, we can try and map to discrete output-classes that indicate if the stock-return was a gain or loss. This is a so-called binary classification problem, and perhaps that is easier for the Neural Network.

### Convert Output Data

First we need to convert the stock-returns into 1 for gains and 0 for losses, and then split into training- and test-sets:

In [ ]:
# df_cls,df_sig_rets

In [ ]:
# %%time
# # Convert stock-returns to binary classes where loss=0 and gain=1
# # In the previous tutorial we just used the sign directly, but it
# # is easier in TensorFlow / Keras to use class-values of 0 and 1.
# df_cls = (np.sign(df_sig_rets[TOTAL_RETURN_1_3Y]) + 1) * 0.5

# # Split the classes into training- and test-sets.
# y_train_cls = df_cls.loc[tickers_train]
# y_test_cls = df_cls.loc[tickers_test]

In [ ]:
%%time
# Convert stock-returns to binary classes where loss=0 and gain=1
# In the previous tutorial we just used the sign directly, but it
# is easier in TensorFlow / Keras to use class-values of 0 and 1.
# df_cls = (np.sign(df_all5['Close_1y']-df_all5['Close']) + 1) * 0.5

df_cls = df_all5['Close_1y']/df_all5['Close']
          
# Split the classes into training- and test-sets.
y_train_cls = df_cls.loc[tickers_train]
y_test_cls = df_cls.loc[tickers_test]

In [ ]:
y_train_cls.shape,y_test_cls.shape,np.unique(df_cls)

In [ ]:
df_all5['Close_1y'].describe()

In [ ]:
df_cls.describe()

In [ ]:
# #7 classes
# conditions = [
#     (df_sig_rets[TOTAL_RETURN_1_3Y] < (-0.4)),
#     (df_sig_rets[TOTAL_RETURN_1_3Y]>=(-0.4)) & (df_sig_rets[TOTAL_RETURN_1_3Y]<(-0.2)),
#     (df_sig_rets[TOTAL_RETURN_1_3Y]>=(-0.2)) & (df_sig_rets[TOTAL_RETURN_1_3Y]<(-0.1)),
#     (df_sig_rets[TOTAL_RETURN_1_3Y]>=(-0.1)) & (df_sig_rets[TOTAL_RETURN_1_3Y]<0.1),
#     (df_sig_rets[TOTAL_RETURN_1_3Y]>=0.1) & (df_sig_rets[TOTAL_RETURN_1_3Y]<0.2),
#     (df_sig_rets[TOTAL_RETURN_1_3Y]>=0.2) & (df_sig_rets[TOTAL_RETURN_1_3Y]<0.4),
#     (df_sig_rets[TOTAL_RETURN_1_3Y]>= 0.4)]
# choices = [0,1,2,3,4,5,6]
# df_sig_rets['return3y'] = np.select(conditions, choices)
# df_cls = df_sig_rets['return3y']
# #print(df_sig_rets)
# np.unique(df_cls)

In [ ]:
# #3 classes
# conditions = [
#     (df_sig_rets[TOTAL_RETURN_1_3Y] < (-0.2)),
#     (df_sig_rets[TOTAL_RETURN_1_3Y]>=(-0.2)) & (df_sig_rets[TOTAL_RETURN_1_3Y]<0.2),
#     (df_sig_rets[TOTAL_RETURN_1_3Y]>=0.2)]
# choices = [0,1,2]
# df_sig_rets['return3y'] = np.select(conditions, choices)
# df_cls = df_sig_rets['return3y']
# #print(df_sig_rets)
# np.unique(df_cls)

In [ ]:
#3 classes
conditions = [
    (df_sig_rets[TOTAL_RETURN_1_1Y] < (-0.1)),
    (df_sig_rets[TOTAL_RETURN_1_1Y]>=(-0.1)) & (df_sig_rets[TOTAL_RETURN_1_1Y]<0.2),
    (df_sig_rets[TOTAL_RETURN_1_1Y]>=0.2)]
choices = [0,1,2]
df_sig_rets['return1y'] = np.select(conditions, choices)
df_cls = df_sig_rets['return1y']
#print(df_sig_rets)
np.unique(df_cls)

In [ ]:
#3 classes
conditions = [
    (df_cls < 1.16),
    ((df_cls>=1.16) & (df_cls<1.34)),
    (df_cls>=1.59)]
choices = [0,1,2]
df_all5['ratio1y'] = np.select(conditions, choices)
df_cls2 = df_all5['ratio1y']
#print(df_sig_rets)
np.unique(df_cls2)

In [ ]:
np.unique(np.select(conditions, choices))

In [ ]:
# Split the classes into training- and test-sets.
y_train_cls = df_cls2.loc[tickers_train]
y_test_cls = df_cls2.loc[tickers_test]

In [ ]:
df_cls2

In [ ]:
# np.unique(df_sig_rets['return3y']), np.unique(y_train_cls)

In [ ]:
np.unique(df_all5['ratio1y'])

In [ ]:
df_cls

In [ ]:
y_train_cls

Because this is a binary classification problem (the result is either a gain or a loss), we could build a Neural Network to use `y_train_cls` directly. But instead we want to build a more general type of Neural Network which supports any number of output classes. For this we must convert the output to so-called one-hot encoded arrays, so a class of 0 (loss) becomes the array `[1,0]` and a class of 1 (gain) becomes the array `[0,1]`.

In [ ]:
to_categorical?

In [ ]:
# Convert classes to one-hot-encoded arrays.
y_train_one_hot = to_categorical(y=y_train_cls.values,
                                 num_classes=3)

# Show the result.
y_train_one_hot

In [ ]:
y_train_cls.values

The training-set is also biased because it contains many more samples with gains compared to losses:

In [ ]:
# Fraction of the training-set that is classified as gains.
gain_fraction = (y_train_cls == 1.0).sum() / len(y_train_cls)
loss_fraction = 1.0 - gain_fraction

# Show it.
gain_fraction

When we train the Neural Network further below, we need to provide new class-weights to counter this imbalance between the two classes:

In [ ]:
y_train_cls[y_train_cls==0]

In [ ]:
compute_class_weight?

In [ ]:
# Calculate the class-weights.
class_weight = compute_class_weight('balanced',
                                    y=y_train_cls,
                                    classes=[0,1,2])

# Show them.
class_weight

We can check that when we multiply these weights by the class-fractions for losses and gains, they both end up with the same weights:

In [ ]:
# class_weight * [loss_fraction, gain_fraction]

In [ ]:
# The problem is that the sklearn API returns a numpy array but the keras requires a dictionary as an input for class_weight (see here). 
# https://stackoverflow.com/questions/61261907/on-colab-class-weight-is-causing-a-valueerror-the-truth-value-of-an-array-wit
class_weight=dict(enumerate(class_weight.flatten(), 0))

In [ ]:
class_weight

### Build and Train Neural Network: Classification

Now that we have prepared the classification data, we can proceed to build the Neural Network. We will use the same overall design as for the regression model above, with minor changes for the classification task.

In [ ]:
X_train

In [ ]:
fit_args = \
{
    # For efficiency, the model is trained on batches of data.
    'batch_size': 4096,
    
    # Number of iterations aka. epochs over the training-set.
    'epochs': 20,
    
    # Fraction of the training-set used for validation after
    # each training-epoch, to assess how well the model performs
    # on unseen data.
    'validation_split': 0.2,

    # Show status during training.
    'verbose': 1,
}

In [ ]:
# The activation function applied after each layer.
# It is important to use a non-linear function, otherwise
# the Neural Network cannot learn non-linear mappings.
activation = 'relu'
#activation = 'selu'

In [ ]:
# Create a new Keras model.
model_clf = keras.Sequential()

# Add an input layer for the signals.
# Note how we set the dimensionality of the input here.
model_clf.add(InputLayer(input_shape=(num_signals,)))

# Add several dense aka. fully-connected layers.
# You can experiment with different designs.
model_clf.add(Dense(128, activation=activation))
model_clf.add(Dense(64, activation=activation))
model_clf.add(Dense(32, activation=activation))
model_clf.add(Dense(16, activation=activation))
model_clf.add(Dense(8, activation=activation))

# Add a layer for the output of the Neural Network.
# This has 2 outputs, one for each possible class, and it uses
# the softmax activation function for classification.
model_clf.add(Dense(3, activation='softmax'))

# Compile the model but don't train it yet.
# Note that it uses a different loss-function and metric
# than the regression-model above.
model_clf.compile(loss='categorical_crossentropy',
                  metrics=['accuracy'],
                  optimizer=RMSprop(0.001))

# Show the model.
model_clf.summary()

In [ ]:
# This model reaches 0.45-0.46 of validation accuracy at epoch=20, 0.47-0.48 at epoch=30~40

# Create a new Keras model.
model_clf = keras.Sequential()

# Add an input layer for the signals.
# Note how we set the dimensionality of the input here.
model_clf.add(InputLayer(input_shape=(num_signals,)))

#using LeakyReLU
from keras.layers import LeakyReLU
# here change your line to leave out an activation 
model_clf.add(Dense(128))
# now add a ReLU layer explicitly:
model_clf.add(LeakyReLU(alpha=0.05))
#model_clf.add(Dropout(0.2))

# add dropout helps
from keras.layers import Dropout
# Add several dense aka. fully-connected layers.
# You can experiment with different designs.
model_clf.add(Dense(128, activation=activation))
model_clf.add(Dropout(0.3))
model_clf.add(Dense(64, activation=activation))
model_clf.add(Dropout(0.3))
model_clf.add(Dense(32, activation=activation))
model_clf.add(Dropout(0.2))
model_clf.add(Dense(16, activation=activation))
model_clf.add(Dropout(0.2))
model_clf.add(Dense(8, activation=activation))
#model_clf.add(Dropout(0.2))

# Add a layer for the output of the Neural Network.
# This has 2 outputs, one for each possible class, and it uses
# the softmax activation function for classification.
model_clf.add(Dense(3, activation='softmax'))

# Compile the model but don't train it yet.
# Note that it uses a different loss-function and metric
# than the regression-model above.
model_clf.compile(loss='categorical_crossentropy',
                  metrics=['accuracy'],
                  optimizer=RMSprop(0.001))

# Show the model.
model_clf.summary()

We can now train the model. This takes a minute or two on an ordinary computer:

In [ ]:
%%time
history_clf = model_clf.fit(x=X_train.values, y=y_train_one_hot,
                            class_weight=class_weight, **fit_args)

We can then plot the training-progress as the classification accuracy which is supposed to be maximized. After 40 epochs through the training-set, the model has learned to map from signals to stock gain/loss for the training-set, with an accuracy of about 96% (this is not for the entire training-set, but only for one or more batches of data).

Unfortunately, the performance on the validation-set gets worse during optimization and ends with an accuracy of about 60%, so the model is clearly not learning the underlying relation between signals and stock gain/loss, but it has instead learned peculiarities and noise in the training-data.

In [ ]:
# Convert training history to a DataFrame.
df = pd.DataFrame(history_clf.history)

# Rename the relevant columns.
names = {'accuracy': 'Training Accuracy',
         'val_accuracy': 'Validation Accuracy'}
df = df[['accuracy', 'val_accuracy']].rename(columns=names)

# Plot the results.
df.plot()

In [ ]:
history_clf

In [ ]:
pd.DataFrame(history_clf.history)

### Classification Results

We can then use the fitted model to predict the classes for the training- and test-sets:

In [ ]:
%%time
y_train_pred_cls = model_clf.predict(X_train.values)
y_test_pred_cls = model_clf.predict(X_test.values)

The Neural Network has 2 outputs which is a likelihood score for each of the two classes:

In [ ]:
y_test_pred_cls

In [ ]:
np.argmax(y_test_pred_cls, axis=1)

Let us convert this to a single integer class-number, by taking the so-called argmax, which is the index of the largest element in each row:

In [ ]:
# Convert one-hot-encoded arrays to integer class-numbers.
y_train_pred_cls = y_train_pred_cls.argmax(axis=1)
y_test_pred_cls = y_test_pred_cls.argmax(axis=1)

# Show the result.
y_test_pred_cls

In [ ]:
np.unique(y_test_cls),np.unique(y_test_pred_cls),np.unique(y_train_one_hot)

In [ ]:
np.unique(y_test_pred_cls),np.unique(y_train_cls),np.unique(y_test_pred_cls)

We can now calculate the classification accuracy for the entire training-set, which is almost 90%, so the model has learned to map financial signals to gain/loss classifications for 1-3 year investment periods:

In [ ]:
accuracy_score(y_true=y_train_cls, y_pred=y_train_pred_cls)

Unfortunately this performance does not generalize to the test-set, which the model has not seen during training. Here the classification accuracy is much lower:

In [ ]:
accuracy_score(y_true=y_test_cls, y_pred=y_test_pred_cls)

This is even worse than simply guessing that the stock-returns are always gains, which would give a classification accuracy of about 70%:

In [ ]:
# Fraction of the test-set that is classified as gains.
(y_test_cls == 2).sum() / len(y_test_cls)

### Confusion Matrix

We can also plot a so-called Confusion Matrix, which shows how well the classes were predicted. First we need a small helper-function for making the plot:

In [ ]:
# def plot_confusion_matrix(y_true, y_pred):
#     """
#     Plot a classification confusion matrix.
    
#     :param y_true: Array of true classes.
#     :param y_pred: Array of predicted classes.
#     """
    
#     # Class labels.
#     labels = [1.0, 0.0]
#     labels_text = ['Gain', 'Loss']

#     # Create confusion matrix.
#     mat = confusion_matrix(y_true=y_true, y_pred=y_pred,
#                            labels=labels)
    
#     # Normalize so all matrix entries sum to 1.0
#     mat = mat / len(y_true)
    
#     # Plot the matrix as a heatmap.
#     sns.heatmap(mat, robust=True,
#                 xticklabels=labels_text,
#                 yticklabels=labels_text)

In [ ]:
# # we now have 5 classes
# def plot_confusion_matrix(y_true, y_pred):
#     """
#     Plot a classification confusion matrix.
    
#     :param y_true: Array of true classes.
#     :param y_pred: Array of predicted classes.
#     """
    
#     # Class labels.
#     labels = [0,1,2,3,4,5,6]
#     labels_text = ['bigerloss','bigloss', 'Loss','nochange',"gain","biggain","biggergain"]

#     # Create confusion matrix.
#     mat = confusion_matrix(y_true=y_true, y_pred=y_pred,
#                            labels=labels)
    
#     # Normalize so all matrix entries sum to 1.0
#     mat = mat / len(y_true)
    
#     # Plot the matrix as a heatmap.
#     sns.heatmap(mat, robust=True,
#                 xticklabels=labels_text,
#                 yticklabels=labels_text)

In [ ]:
#  3 classes
def plot_confusion_matrix(y_true, y_pred):
    """
    Plot a classification confusion matrix.
    
    :param y_true: Array of true classes.
    :param y_pred: Array of predicted classes.
    """
    
    # Class labels.
    labels = [0,1,2]
    labels_text = ['Loss','modest_gain',"high_gain"]

    # Create confusion matrix.
    mat = confusion_matrix(y_true=y_true, y_pred=y_pred,
                           labels=labels)
    
    # Normalize so all matrix entries sum to 1.0
    mat = mat / len(y_true)
    
    # Plot the matrix as a heatmap.
    sns.heatmap(mat, robust=True,
                xticklabels=labels_text,
                yticklabels=labels_text)

For the training-set, we again see that the model has learned to accurately classify gains and losses. Note how the colours are different, because the training-set contains many more gains than losses.

In [ ]:
y_train_cls,y_train_pred_cls

In [ ]:
plot_confusion_matrix(y_true=y_train_cls, y_pred=y_train_pred_cls)

The confusion matrix for the test-set, shows that many classes were correctly predicted as gains, and there were a lot of errors in the rest of the predictions:

In [ ]:
plot_confusion_matrix(y_true=y_test_cls, y_pred=y_test_pred_cls)

### Classification Examples

We can also try and plot the model's predicted versus actual gain/loss classification for a stock in the training-set. As we can see, the model has learned this mapping quite well:

In [ ]:
stock_index=np.where(tickers_train=='IBM')
print(stock_index)

In [ ]:
# Create a DataFrame with actual and predicted classifications.
# This is for the training-set.
df_y_train_cls = pd.DataFrame(y_train_cls)
df_y_train_cls['ratio_1y_PRED'] = y_train_pred_cls

# Plot the actual and predicted stock-returns for the first stock.
ticker = tickers_train[0]
_ = df_y_train_cls.loc[ticker].plot(title=ticker)

In [ ]:
df_y_train_cls.loc[ticker]

Now let us try and plot the model's predicted versus actual gain/loss classifications for the first stock in the test-set. As we can see, the model is quite bad at predicting this for data that it has not seen during its training:

In [ ]:
# Create a DataFrame with actual and predicted classifications.
# This is for the test-set.
df_y_test_cls = pd.DataFrame(y_test_cls)
df_y_test_cls['ratio_1y_PRED'] = y_test_pred_cls

# Plot the actual and predicted stock-returns for the first stock.
ticker = tickers_test[11]
_ = df_y_test_cls.loc[ticker].plot(title=ticker)

In [ ]:
df_y_test_cls.loc[ticker]

In [ ]:
# accuracy_score for one stock only
accuracy_score(y_true=df_y_test_cls.loc[ticker]['ratio1y'], y_pred=df_y_test_cls.loc[ticker]['ratio_1y_PRED'])

## Discussion

We used a Neural Network to learn the relation between signals and future stock-returns. This worked reasonably well on the training-data, but it did not generalize to the test-data, where the Neural Network performed worse than just guessing. In fact, the result was even worse than the previous tutorial which used a Machine Learning (ML) model called a Random Forest.

The problem might be the design of the Neural Network, that is, the number and size of the layers, their activation functions, etc. You can try and use so-called Hyper-Parameter Optimization such as the [Keras Tuner](https://keras-team.github.io/keras-tuner/), to try and find a better design for the Neural Network.

But we have now seen two very different ML models having the same problem with this dataset, so perhaps **it means that these financial signals simply cannot be used to predict future stock-returns for 1-3 year investment periods.**


### Machine Learning vs. Good Science

In the previous tutorial, we had a brief discussion about Machine Learning versus Good Science. We made an important point that is worth repeating here.

In general, these experiments show that Machine Learning cannot replace good science. *Even if the ML model had performed well on the test-data, it would not have provided an explanation for the underlying relation between signals and stock-returns, so we would not know whether it was truly something that would work in the future. To explain this we need proper scientific reasoning.*

A good example of scientific reasoning is the discovery and formulation of the Law of Gravity. A simple experiment would be to drop various items from different heights and take lots of measurements of how fast the items fall to the ground.

You can then make a noisy scatter-plot that shows the relation between item-size, item-weight, drop-height, and drop-speed. Statistical analysis would be able to give you a rough idea whether there is some relation in the data and if it is statistically significant, so the observed data is probably not just due to random chance. 

Machine Learning takes this one step further, in that it might be able to create a mathematical or computational model for the non-linear relation in the data, so you could use the ML model to predict the drop-speed if you input the other data. **But the ML model would not be able to explain *why* this relation exists in the data.** For this we need a proper theoretical framework such as Classical Mechanics in the case of the Law of Gravity.


### Predicting Long-Term Stock Returns

When it comes to predicting long-term stock-returns, there are actually only 3 factors that matter: <font color='red'> **The future dividends, sales-growth, and change in the P/Sales valuation ratio.** </font>(You could instead use earnings-growth or book-value growth, and the P/E or P/Book valuation ratios.) If we can predict the future dividend, sales-growth and P/Sales ratio, then we can predict the future stock-return.

When making scatter-plots of historical P/Sales ratios versus long-term stock-returns, we often see a particular downwards-sloping curve, and it turns out there is a certain formula for this curve, which is derived from the mathematical definition of Annualized Return, see [here](https://github.com/Hvass-Labs/FinanceOps/blob/master/01C_Theory_of_Long-Term_Stock_Forecasting.ipynb). The formula takes as input the average dividend yield, sales-growth and P/Sales ratios that you predict for the future, and produces a curve that can be used to predict future stock-returns from these 3 factors and the P/Sales ratio today. The accuracy of the prediction depends on how well you have predicted these 3 factors.

Machine Learning cannot uncover and explain such relations, whether it is the Law of Gravity, or the formula for long-term stock-returns. Human reasoning is still needed for this.

## License (MIT)

This is published under the
[MIT License](https://github.com/simfin/simfin-tutorials/blob/master/LICENSE.txt)
which allows very broad use for both academic and commercial purposes.

You are very welcome to modify and use this source-code in your own project. Please keep a link to the [original repository](https://github.com/simfin/simfin-tutorials).
